In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
from random import seed

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import  MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Imputer

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM, BatchNormalization
from keras import layers
from keras import optimizers
from keras import callbacks
from keras import models

Using TensorFlow backend.


In [2]:
#Generate Dataset
#Airfoil Self-Noise
df_airfoil=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Airfoil_self_noise.csv')

#Automobile
df_automobile=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Automobile_data.txt')

#Autompg
df_autompg=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Auto-mpg.csv')

#ChallengerUSA
df_challenger=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Challenger USA Space Shuttle O-Ring.csv')

#Combine Cycle Power
df_combine=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Combine Cycle Power Plant.csv')

#Computer Hardware
df_computer=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Computer Hardware.csv')

#Concrete Compressive Strength
df_concrete_compressive_strength=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Compressive Strength.csv')

#Concrete Slump Test
df_concrete1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')
df_concrete2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')
df_concrete3=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')

#Energy Efficiency
df_energy1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')

#Fertility Diagnosis
df_fertility=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\fertility_Diagnosis.csv')

#Forest Fires
df_forest=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\forestfires.csv')

#Housing
df_housing=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Housing.csv')

#Istanbul Stock Exchange
df_istanbul=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Istanbul Stock Exchange.csv')

#WDBC
df_wdbc=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\wdbc.csv')

In [3]:
#remove Nan columns of df_energy1
df_energy1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy1.drop(df_energy1.columns[11],inplace=True, axis = 1)
df_energy1.drop(df_energy1.columns[10],inplace=True, axis = 1)

#remove Nan columns of df_energy2
df_energy2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy2.drop(df_energy2.columns[11],inplace=True, axis = 1)
df_energy2.drop(df_energy2.columns[10],inplace=True, axis = 1)

#remove question marks
#df_automobile
imp= Imputer(missing_values="NaN", strategy="mean")
df_automobile=df_automobile.replace("?", np.NaN)
df_automobile['num-of-doors'] = df_automobile['num-of-doors'].fillna("four")
for i in df_automobile.columns:
    if df_automobile[i].isnull().sum()>0:
        df_automobile[i]=imp.fit_transform(df_automobile[[i]])
#df_automobile = df_automobile[(df_automobile.astype(str) != '?').all(axis=1)]
#df_automobile.reset_index(drop=True, inplace=True)
df_automobile = df_automobile.drop(['engine-location'], axis=1)
#df_autompg
df_autompg = df_autompg[(df_autompg.astype(str) != '?').all(axis=1)]
df_autompg.reset_index(drop=True, inplace=True)

#convert string to number
le=LabelEncoder()
#df_automobile
string_automobile=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=string_automobile.apply(le.fit_transform)
encoder=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']].astype(np.int64)
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=encoder
a=df_automobile[['normalized-losses','horsepower','price']].astype(np.int64)
b=df_automobile[['bore','stroke','peak-rpm']].astype(float)
df_automobile[['normalized-losses','horsepower','price']]=a
df_automobile[['bore','stroke','peak-rpm']]=b
#df_computer
string_computer=df_computer[['Vendor','Model']]
df_computer[['Vendor','Model']]=string_computer.apply(le.fit_transform)
df_computer[['Vendor','Model']]=df_computer[['Vendor','Model']].astype(np.int64)

#dropna spesific column & special preprocessing
#df_forest
df_forest=df_forest[(df_forest['area'].astype(int) != 0)] 
df_forest.reset_index(drop=True, inplace=True)
num_features=10
dimension=(num_features+1)*9+10
population_size=(2*dimension+1)*2
df_forest=df_forest[:population_size]

C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
def preprocessing_data1(df,num_features):
    dimension=(num_features+1)*9+10
    population_size=(2*dimension+1)*2
    df=df.dropna()
    df=df[:population_size]
    df
    return df

In [5]:
def feature_selection(df):
    cor=df.corr()
    plt.figure(figsize=(100,50))
    sns.set(font_scale=3)
    sns.heatmap(cor, annot=True, cmap=plt.cm.Blues)
    plt.show()
    return cor

In [6]:
def train_test_split1(x, y):
    x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.15, random_state=42)
    return x_train, x_test, y_train, y_test

In [7]:
def preprocessing_data2(df):
    train_size = 0.85*len(df)
    train_size = int(train_size)
    df_train = df[:train_size]
    df_test = df[train_size:]
    print(len(df_test))
    print(len(df_train))
    validation_size=len(df_test)/len(df_train)
    print(validation_size)
    return df_train, df_test, validation_size 

In [8]:
#df_airfoil=preprocessing_data1(df_airfoil,5)
#df_automobile=preprocessing_data1(df_automobile,22)
#df_autompg=preprocessing_data1(df_autompg,7)
#df_challenger=preprocessing_data1(df_challenger,3)
#df_combine=preprocessing_data1(df_combine,4)
#df_computer=preprocessing_data1(df_computer,9)
#df_concrete_compressive_strength=preprocessing_data1(df_concrete_compressive_strength,8)
#df_concrete1=preprocessing_data1(df_concrete1,7)
#df_concrete2=preprocessing_data1(df_concrete2,7)
#df_concrete3=preprocessing_data1(df_concrete3,7)
#df_energy1=preprocessing_data1(df_energy1,8)
#df_energy2=preprocessing_data1(df_energy2,8)

#df_fertility=preprocessing_data1(df_fertility,9)
for i in df_fertility:
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'N'] = 1
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'O'] = 2
df_fertility['Diagnosis']=df_fertility['Diagnosis'].astype(np.int64)

#df_forest=preprocessing_data1(df_forest,10)
#df_housing=preprocessing_data1(df_housing,13)

#df_istanbul=preprocessing_data1(df_istanbul,7)
new_header = df_istanbul.iloc[0]
df_istanbul = df_istanbul[1:]
df_istanbul.columns = new_header
df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']]=df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']].astype(np.float64)


#df_wdbc=preprocessing_data1(df_wdbc,10)
df_wdbc.drop(df_wdbc.columns[0], axis=1, inplace = True)
df_wdbc.columns=['id','diagnosis','radius_mean','texture_mean','perimeter_mean','area_mean','smoothness_mean','compactness_mean','concavity_mean','concave points_mean','symmetry_mean','fractal_dimension_mean','radius_se','texture_se','perimeter_se','area_se','smoothness_se','compactness_se','concavity_se','concave points_se','symmetry_se','fractal_dimension_se','radius_worst','texture_worst','perimeter_worst','area_worst','smoothness_worst','compactness_worst','concavity_worst','concave points_worst','symmetry_worst','fractal_dimension_worst']
for i in df_wdbc:
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'M'] = 1
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'B'] = 2
df_wdbc['diagnosis']=df_wdbc['diagnosis'].astype(np.object)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a 

In [9]:
print(df_airfoil.head())
print(df_autompg.head())
print(df_automobile.head())
print(df_challenger.head())

   Frequency  Angle of Attack  Chord length  Free-stream velocity  \
0        800              0.0        0.3048                  71.3   
1       1000              0.0        0.3048                  71.3   
2       1250              0.0        0.3048                  71.3   
3       1600              0.0        0.3048                  71.3   
4       2000              0.0        0.3048                  71.3   

   Suction side displacement thickness  Scaled sound pressure level  
0                             0.002663                      126.201  
1                             0.002663                      125.201  
2                             0.002663                      125.951  
3                             0.002663                      127.591  
4                             0.002663                      127.461  
    Mpg  Cylinders  Displacement Horsepower  Weight  Acceleration  Model Year  \
0  18.0          8         307.0        130    3504          12.0          70   
1  

In [10]:
#Train and Test 17 datasets
#Airfoil
x_airfoil = df_airfoil.drop(['Scaled sound pressure level'], axis = 1)
y_airfoil = df_airfoil['Scaled sound pressure level']
#Wdbc
x_wdbc = df_wdbc[['radius_mean','perimeter_mean','area_mean','concavity_mean','concave points_mean','radius_worst','perimeter_worst','area_worst','concave points_worst']]
y_wdbc = df_wdbc['diagnosis']
labelencoder_Y = LabelEncoder()
y_wdbc = labelencoder_Y.fit_transform(y_wdbc)
#Automobile
x_automobile = df_automobile.drop(['price','make','body-style'], axis = 1)
y_automobile = df_automobile['price']
y_automobile = np.reshape(y_automobile.values, (-1,1))
#Autompg
x_autompg = df_autompg.drop(['Mpg','Car Name'], axis = 1)
y_autompg = df_autompg['Mpg']
#Challenger
x_challenger = df_challenger.drop(['Number of O-rings','Temporal Order of Flight'], axis = 1)
y_challenger = df_challenger['Number of O-rings']
#Combine
x_combine = df_combine.drop(['PE'], axis = 1)
y_combine = df_combine['PE']
y_combine = np.reshape(y_combine.values, (-1,1))
#Computer
x_computer = df_computer.drop(['ERP'], axis = 1)
y_computer = df_computer['ERP']
y_computer = np.reshape(y_computer.values, (-1,1))
#Concrete_compressive_strength
x_concrete_compressive_strength = df_concrete_compressive_strength.drop(['CCS'], axis = 1)
y_concrete_compressive_strength = df_concrete_compressive_strength['CCS']
y_concrete_compressive_strength = np.reshape(y_concrete_compressive_strength.values, (-1,1))
#Concrete1
x_concrete1 = df_concrete1.drop(['No','Compressive Strength (28-day)(Mpa)'], axis = 1)
y_concrete1 = df_concrete1['Compressive Strength (28-day)(Mpa)']
#Concrete2
x_concrete2 = df_concrete2.drop(['No','SLUMP(cm)'], axis = 1)
y_concrete2 = df_concrete2['SLUMP(cm)']
y_concrete2 = np.reshape(y_concrete2.values, (-1,1))
#Concrete3
x_concrete3 = df_concrete3.drop(['No','FLOW(cm)'], axis = 1)
y_concrete3 = df_concrete3['FLOW(cm)']
y_concrete3 = np.reshape(y_concrete3.values, (-1,1))
#Energy1
x_energy1 = df_energy1.drop(['Y1'], axis = 1)
y_energy1 = df_energy1['Y1']
#Energy2
x_energy2 = df_energy2.drop(['Y2'], axis = 1)
y_energy2 = df_energy2['Y2']
#Fertility
x_fertility = df_fertility.drop(['Diagnosis'], axis = 1)
y_fertility = df_fertility['Diagnosis']
#Forest
x_forest = df_forest.drop(['month','day','area'], axis = 1)
y_forest = df_forest['area']
y_forest = np.reshape(y_forest.values, (-1,1))
#Housing
x_housing = df_housing.drop(['MEDV'], axis = 1)
y_housing = df_housing['MEDV']
#Istanbul
x_istanbul = df_istanbul.drop(['date','ISE1','ISE2'], axis = 1)
y_istanbul = df_istanbul[['ISE1','ISE2']]
y_istanbul['mean'] = y_istanbul.mean(axis=1)
y_istanbul = y_istanbul['mean']

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
x_train_airfoil, x_test_airfoil, y_train_airfoil, y_test_airfoil = train_test_split1(x_airfoil, y_airfoil)
x_train_automobile, x_test_automobile, y_train_automobile, y_test_automobile = train_test_split1(x_automobile, y_automobile)
x_train_autompg, x_test_autompg, y_train_autompg, y_test_autompg = train_test_split1(x_autompg, y_autompg)
x_train_challenger, x_test_challenger, y_train_challenger, y_test_challenger = train_test_split1(x_challenger, y_challenger)
x_train_combine, x_test_combine, y_train_combine, y_test_combine = train_test_split1(x_combine, y_combine)
x_train_computer, x_test_computer, y_train_computer, y_test_computer = train_test_split1(x_computer, y_computer)
x_train_concrete_compressive_strength, x_test_concrete_compressive_strength, y_train_concrete_compressive_strength, y_test_concrete_compressive_strength = train_test_split1(x_concrete_compressive_strength, y_concrete_compressive_strength)
x_train_concrete1, x_test_concrete1, y_train_concrete1, y_test_concrete1 = train_test_split1(x_concrete1, y_concrete1)
x_train_concrete2, x_test_concrete2, y_train_concrete2, y_test_concrete2 = train_test_split1(x_concrete2, y_concrete2)
x_train_concrete3, x_test_concrete3, y_train_concrete3, y_test_concrete3 = train_test_split1(x_concrete3, y_concrete3)
x_train_energy1, x_test_energy1, y_train_energy1, y_test_energy1 = train_test_split1(x_energy1, y_energy1)
x_train_energy2, x_test_energy2, y_train_energy2, y_test_energy2 = train_test_split1(x_energy2, y_energy2)
x_train_fertility, x_test_fertility, y_train_fertility, y_test_fertility = train_test_split1(x_fertility, y_fertility)
x_train_forest, x_test_forest, y_train_forest, y_test_forest = train_test_split1(x_forest, y_forest)
x_train_housing, x_test_housing, y_train_housing, y_test_housing = train_test_split1(x_housing, y_housing)
x_train_istanbul, x_test_istanbul, y_train_istanbul, y_test_istanbul = train_test_split1(x_istanbul, y_istanbul)
x_train_wdbc, x_test_wdbc, y_train_wdbc, y_test_wdbc = train_test_split1(x_wdbc, y_wdbc)

In [12]:
def train_val_split(x_train, y_train):
    X_train, X_val, Y_train, Y_val= train_test_split(x_train, y_train, test_size=0.176, random_state=42)
    return X_train, X_val, Y_train, Y_val

In [13]:
def standard_scaler(X_train, X_val, x_test):
    sc=StandardScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=sc.fit(X_train)
    #X_scale_val=sc.fit(X_val)
    #x_scale_test=sc.fit(x_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_train.transform(X_val)
    x_test=X_scale_train.transform(x_test)
    return X_train, X_val, x_test, X_scale_train

In [14]:
def min_max_scaler_automobile(X_train, X_val, x_test, Y_train, Y_val, y_test):
    minmax_x=MinMaxScaler()
    minmax_y=MinMaxScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=minmax_x.fit(X_train)
    #X_scale_val=min_max.fit(X_val)
    #x_scale_test=min_max.fit(x_test)
    Y_scale_train=minmax_y.fit(Y_train)
    #Y_scale_val=min_max.fit(Y_val)
    #y_scale_test=min_max.fit(y_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_train.transform(X_val)
    x_test=X_scale_train.transform(x_test)
    Y_train=Y_scale_train.transform(Y_train)
    Y_val=Y_scale_train.transform(Y_val)
    y_test=Y_scale_train.transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test, X_scale_train, Y_scale_train

In [15]:
def min_max_scaler(X_train, X_val, x_test, Y_train, Y_val, y_test):
    min_max=MinMaxScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=min_max.fit(X_train)
    X_scale_val=min_max.fit(X_val)
    x_scale_test=min_max.fit(x_test)
    Y_scale_train=min_max.fit(Y_train)
    Y_scale_val=min_max.fit(Y_val)
    y_scale_test=min_max.fit(y_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_val.transform(X_val)
    x_test=x_scale_test.transform(x_test)
    Y_train=Y_scale_train.transform(Y_train)
    Y_val=Y_scale_train.transform(Y_val)
    y_test=y_scale_test.transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test, min_max

In [16]:
def normalization(X_train, X_val, x_test, Y_train, Y_val, y_test):
    normalizer=Normalizer()
    X_train=normalizer.fit_transform(X_train)
    X_val=normalizer.fit_transform(X_val)
    x_test=normalizer.fit_transform(x_test)
    Y_train=normalizer.fit_transform(Y_train)
    Y_val=normalizer.fit_transform(Y_val)
    y_test=normalizer.fit_transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test

In [17]:
#Optimizer list
adam0 = optimizers.Adam(lr=0.1)
adam1 = optimizers.Adam(lr=0.001)
adam2 = optimizers.Adam(lr=0.01)
adam3 = optimizers.Adam(lr=0.0001)
adam4 = optimizers.Adam(lr=0.00001)
sgd = optimizers.SGD(lr=0.00001, decay=1e-4, momentum=0.9, nesterov=True)
RMSprop= optimizers.RMSprop(lr=0.01,rho=0.9)
adagrad = optimizers.Adagrad(lr=0.01)
adadelta = optimizers.Adadelta(lr=1.0)

In [18]:
def NN_model_structure_regression(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [19]:
X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil = train_val_split(x_train_airfoil, y_train_airfoil)
X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile = train_val_split(x_train_automobile, y_train_automobile)
X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg = train_val_split(x_train_autompg, y_train_autompg)
X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger = train_val_split(x_train_challenger, y_train_challenger)
X_train_combine, X_val_combine, Y_train_combine, Y_val_combine = train_val_split(x_train_combine, y_train_combine)
X_train_computer, X_val_computer, Y_train_computer, Y_val_computer = train_val_split(x_train_computer, y_train_computer)
X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength = train_val_split(x_train_concrete_compressive_strength, y_train_concrete_compressive_strength)
X_train_concrete1, X_val_concrete1, Y_train_concrete1, Y_val_concrete1 = train_val_split(x_train_concrete1, y_train_concrete1)
X_train_concrete2, X_val_concrete2, Y_train_concrete2, Y_val_concrete2 = train_val_split(x_train_concrete2, y_train_concrete2)
X_train_concrete3, X_val_concrete3, Y_train_concrete3, Y_val_concrete3 = train_val_split(x_train_concrete3, y_train_concrete3)
X_train_energy1, X_val_energy1, Y_train_energy1, Y_val_energy1 = train_val_split(x_train_energy1, y_train_energy1)
X_train_energy2, X_val_energy2, Y_train_energy2, Y_val_energy2 = train_val_split(x_train_energy2, y_train_energy2)
X_train_fertility, X_val_fertility, Y_train_fertility, Y_val_fertility = train_val_split(x_train_fertility, y_train_fertility)
X_train_forest, X_val_forest, Y_train_forest, Y_val_forest = train_val_split(x_train_forest, y_train_forest)
X_train_housing, X_val_housing, Y_train_housing, Y_val_housing = train_val_split(x_train_housing, y_train_housing)
X_train_istanbul, X_val_istanbul, Y_train_istanbul, Y_val_istanbul = train_val_split(x_train_istanbul, y_train_istanbul)
X_train_wdbc, X_val_wdbc, Y_train_wdbc, Y_val_wdbc = train_val_split(x_train_wdbc, y_train_wdbc)

In [20]:
X_train_airfoil, X_val_airfoil, x_test_airfoil, X_scale_train_airfoil=standard_scaler(X_train_airfoil, X_val_airfoil, x_test_airfoil)
#X_train_automobile, X_val_automobile, x_test_automobile=standard_scaler(X_train_automobile, X_val_automobile, x_test_automobile)
#X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile = normalization(X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile)
X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile, X_scale_train_automobile, Y_scale_train_automobile=min_max_scaler_automobile(X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile)
X_train_autompg, X_val_autompg, x_test_autompg, X_scale_train_autompg=standard_scaler(X_train_autompg, X_val_autompg, x_test_autompg)
X_train_challenger, X_val_challenger, x_test_challenger, X_scale_train_challenger=standard_scaler(X_train_challenger, X_val_challenger, x_test_challenger)
X_train_combine, X_val_combine, x_test_combine, Y_train_combine, Y_val_combine, y_test_combine, min_max_combine=min_max_scaler(X_train_combine, X_val_combine, x_test_combine, Y_train_combine, Y_val_combine, y_test_combine)
X_train_computer, X_val_computer, x_test_computer, Y_train_computer, Y_val_computer, y_test_computer, min_max_computer=min_max_scaler(X_train_computer, X_val_computer, x_test_computer, Y_train_computer, Y_val_computer, y_test_computer)
X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, x_test_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength, y_test_concrete_compressive_strength, min_max_concrete_compressive_strength=min_max_scaler(X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, x_test_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength, y_test_concrete_compressive_strength)
X_train_concrete1, X_val_concrete1, x_test_concrete1, X_scale_train_concrete1=standard_scaler(X_train_concrete1, X_val_concrete1, x_test_concrete1)
X_train_concrete2, X_val_concrete2, x_test_concrete2, Y_train_concrete2, Y_val_concrete2, y_test_concrete2, min_max_concrete2=min_max_scaler(X_train_concrete2, X_val_concrete2, x_test_concrete2, Y_train_concrete2, Y_val_concrete2, y_test_concrete2)
X_train_concrete3, X_val_concrete3, x_test_concrete3, Y_train_concrete3, Y_val_concrete3, y_test_concrete3, min_max_concrete3=min_max_scaler(X_train_concrete3, X_val_concrete3, x_test_concrete3, Y_train_concrete3, Y_val_concrete3, y_test_concrete3)
X_train_energy1, X_val_energy1, x_test_energy1, X_scale_train_energy1=standard_scaler(X_train_energy1, X_val_energy1, x_test_energy1)
X_train_energy2, X_val_energy2, x_test_energy2, X_scale_train_energy2=standard_scaler(X_train_energy2, X_val_energy2, x_test_energy2)
X_train_fertility, X_val_fertility, x_test_fertility, X_scale_train_fertility=standard_scaler(X_train_fertility, X_val_fertility, x_test_fertility)
X_train_forest, X_val_forest, x_test_forest, Y_train_forest, Y_val_forest, y_test_forest, min_max_forest=min_max_scaler(X_train_forest, X_val_forest, x_test_forest, Y_train_forest, Y_val_forest, y_test_forest)
X_train_housing, X_val_housing, x_test_housing, X_scale_train_housing=standard_scaler(X_train_housing, X_val_housing, x_test_housing)
X_train_istanbul, X_val_istanbul, x_test_istanbul, X_scale_train_istanbul=standard_scaler(X_train_istanbul, X_val_istanbul, x_test_istanbul)

In [21]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_2 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________

Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 403us/step - 

Epoch 65/1000
1052/1052 [==============================] - 0s 101us/step - loss: 22.2638 - val_loss: 26.8399
Epoch 66/1000
1052/1052 [==============================] - 0s 98us/step - loss: 22.5665 - val_loss: 25.2136
Epoch 67/1000
1052/1052 [==============================] - 0s 96us/step - loss: 21.8354 - val_loss: 26.5113
Epoch 68/1000
1052/1052 [==============================] - 0s 94us/step - loss: 21.5556 - val_loss: 25.5833
Epoch 69/1000
1052/1052 [==============================] - 0s 90us/step - loss: 21.8314 - val_loss: 26.8919
Epoch 70/1000
1052/1052 [==============================] - 0s 90us/step - loss: 21.6991 - val_loss: 25.5913
Epoch 71/1000
1052/1052 [==============================] - 0s 90us/step - loss: 21.8823 - val_loss: 25.7007
Epoch 72/1000
1052/1052 [==============================] - 0s 87us/step - loss: 21.7511 - val_loss: 26.1961
Epoch 73/1000
1052/1052 [==============================] - 0s 87us/step - loss: 21.8480 - val_loss: 25.4613
Epoch 74/1000
1052/1052 [==

1052/1052 [==============================] - 0s 91us/step - loss: 16.1573 - val_loss: 18.7234
Epoch 216/1000
1052/1052 [==============================] - 0s 90us/step - loss: 15.4456 - val_loss: 18.3364
Epoch 217/1000
1052/1052 [==============================] - 0s 86us/step - loss: 14.8639 - val_loss: 18.5325
Epoch 218/1000
1052/1052 [==============================] - 0s 90us/step - loss: 15.1995 - val_loss: 17.2437
Epoch 219/1000
1052/1052 [==============================] - 0s 90us/step - loss: 14.8773 - val_loss: 17.9720
Epoch 220/1000
1052/1052 [==============================] - 0s 84us/step - loss: 14.9519 - val_loss: 17.2121
Epoch 221/1000
1052/1052 [==============================] - 0s 89us/step - loss: 15.2752 - val_loss: 17.9957
Epoch 222/1000
1052/1052 [==============================] - 0s 88us/step - loss: 15.5759 - val_loss: 16.8675
Epoch 223/1000
1052/1052 [==============================] - 0s 90us/step - loss: 14.5442 - val_loss: 18.0699
Epoch 224/1000
1052/1052 [========

1052/1052 [==============================] - 0s 90us/step - loss: 9.8389 - val_loss: 12.1882
Epoch 366/1000
1052/1052 [==============================] - 0s 88us/step - loss: 9.9376 - val_loss: 11.3456
Epoch 367/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.2065 - val_loss: 11.6979
Epoch 368/1000
1052/1052 [==============================] - 0s 96us/step - loss: 9.8895 - val_loss: 12.2514
Epoch 369/1000
1052/1052 [==============================] - 0s 101us/step - loss: 9.4659 - val_loss: 12.4376
Epoch 370/1000
1052/1052 [==============================] - 0s 96us/step - loss: 9.4025 - val_loss: 11.3865
Epoch 371/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.5601 - val_loss: 11.0904
Epoch 372/1000
1052/1052 [==============================] - 0s 88us/step - loss: 9.4992 - val_loss: 12.0453
Epoch 373/1000
1052/1052 [==============================] - 0s 93us/step - loss: 9.5952 - val_loss: 11.9128
Epoch 374/1000
1052/1052 [================

Epoch 441/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.1124 - val_loss: 11.9510
Epoch 442/1000
1052/1052 [==============================] - 0s 85us/step - loss: 9.5899 - val_loss: 10.5185
Epoch 443/1000
1052/1052 [==============================] - 0s 88us/step - loss: 9.2638 - val_loss: 12.4839
Epoch 444/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.5738 - val_loss: 11.0009
Epoch 445/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.2836 - val_loss: 11.9007
Epoch 446/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.1942 - val_loss: 12.5540
Epoch 447/1000
1052/1052 [==============================] - 0s 87us/step - loss: 10.0482 - val_loss: 11.8322
Epoch 448/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.5136 - val_loss: 11.0587
Epoch 449/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.1388 - val_loss: 10.7833
Epoch 450/1000
1052/1052 [=

...

Epoch 744/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.2820 - val_loss: 10.3062
Epoch 745/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.9815 - val_loss: 11.7752
Epoch 746/1000
1052/1052 [==============================] - 0s 93us/step - loss: 9.1438 - val_loss: 10.8278
Epoch 747/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.7024 - val_loss: 11.7364
Epoch 748/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.8894 - val_loss: 10.7746
Epoch 749/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.8635 - val_loss: 10.4289
Epoch 750/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.9491 - val_loss: 11.0045
Epoch 751/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.8361 - val_loss: 10.9276
Epoch 752/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.9683 - val_loss: 10.9278
Epoch 753/1000
1052/1052 [==

Epoch 820/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.1090 - val_loss: 11.3274
Epoch 821/1000
1052/1052 [==============================] - 0s 95us/step - loss: 8.7375 - val_loss: 13.5114
Epoch 822/1000
1052/1052 [==============================] - 0s 96us/step - loss: 9.3223 - val_loss: 10.5708
Epoch 823/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.9976 - val_loss: 10.8281
Epoch 824/1000
1052/1052 [==============================] - 0s 92us/step - loss: 9.1400 - val_loss: 11.3068
Epoch 825/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.8457 - val_loss: 10.7445
Epoch 826/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.9033 - val_loss: 10.4456
Epoch 827/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.1884 - val_loss: 10.7492
Epoch 828/1000
1052/1052 [==============================] - 0s 85us/step - loss: 9.4624 - val_loss: 11.0390
Epoch 829/1000
1052/1052 [==

Epoch 896/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.7401 - val_loss: 10.4755
Epoch 897/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.1625 - val_loss: 11.5431
Epoch 898/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.9314 - val_loss: 10.7372
Epoch 899/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.2621 - val_loss: 11.2741
Epoch 900/1000
1052/1052 [==============================] - 0s 88us/step - loss: 9.8853 - val_loss: 11.2288
Epoch 901/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.4526 - val_loss: 10.7029
Epoch 902/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.6435 - val_loss: 10.5789
Epoch 903/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.0842 - val_loss: 10.2400
Epoch 904/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.7557 - val_loss: 11.5126
Epoch 905/1000
1052/1052 [==

Epoch 972/1000
1052/1052 [==============================] - 0s 83us/step - loss: 8.7704 - val_loss: 10.2628
Epoch 973/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.5909 - val_loss: 10.4916
Epoch 974/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.9081 - val_loss: 12.0301
Epoch 975/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.8690 - val_loss: 10.6649
Epoch 976/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.0677 - val_loss: 10.3749
Epoch 977/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.6019 - val_loss: 11.2440
Epoch 978/1000
1052/1052 [==============================] - 0s 94us/step - loss: 8.8812 - val_loss: 10.4728
Epoch 979/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.8673 - val_loss: 10.4074
Epoch 980/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.2826 - val_loss: 12.0836
Epoch 981/1000
1052/1052 [==

[array([[-1.9559397 , -0.21596177,  2.366864  , -3.831444  , -3.3035686 ],
        [ 0.34534797, -0.25029755, -0.1838482 ,  0.30868968, -0.03377806],
        [ 0.2943394 , -0.37147492, -0.03003568, -0.6304434 , -0.02251988],
        [ 0.05219527,  0.09726054, -0.1832903 ,  0.16818589, -0.16364615],
        [-0.20100169, -0.18387212,  2.9241974 , -0.43552953, -0.44549966]],
       dtype=float32),
 array([-1.3525501 , -0.09846009,  4.4292083 , -4.7381516 , -4.2068844 ],
       dtype=float32),
 array([[-0.6809769 ,  0.4206214 ,  0.39323035,  0.28940502, -0.5959601 ,
         -1.227106  ,  0.8751311 , -0.8745659 ,  0.7348782 , -0.69360673],
        [-0.808588  ,  1.5924869 ,  0.50369525,  0.6683649 , -1.4414929 ,
         -0.34412235,  0.9877406 , -0.99909794,  0.8753164 , -0.8959788 ],
        [-0.9182221 ,  2.082325  ,  1.4653559 ,  1.5441968 , -1.5932399 ,
         -2.00697   ,  1.1069134 , -2.0278935 ,  2.0587192 , -2.103446  ],
        [ 1.5777928 , -1.7653612 , -1.5855818 , -2.051430

In [22]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_adam.h5')

In [23]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, sgd, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_3 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_4 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 166us/step - l

Epoch 67/1000
1052/1052 [==============================] - 0s 75us/step - loss: 19.1987 - val_loss: 24.0649
Epoch 68/1000
1052/1052 [==============================] - 0s 70us/step - loss: 19.1617 - val_loss: 24.1002
Epoch 69/1000
1052/1052 [==============================] - 0s 72us/step - loss: 19.1851 - val_loss: 24.1726
Epoch 70/1000
1052/1052 [==============================] - 0s 69us/step - loss: 19.1213 - val_loss: 24.0298
Epoch 71/1000
1052/1052 [==============================] - 0s 73us/step - loss: 19.0466 - val_loss: 24.0426
Epoch 72/1000
1052/1052 [==============================] - 0s 65us/step - loss: 19.0484 - val_loss: 23.9414
Epoch 73/1000
1052/1052 [==============================] - 0s 65us/step - loss: 18.9992 - val_loss: 23.9798
Epoch 74/1000
1052/1052 [==============================] - 0s 69us/step - loss: 18.9880 - val_loss: 23.9695
Epoch 75/1000
1052/1052 [==============================] - 0s 72us/step - loss: 18.9609 - val_loss: 23.9599
Epoch 76/1000
1052/1052 [===

1052/1052 [==============================] - 0s 74us/step - loss: 17.0829 - val_loss: 21.4694
Epoch 218/1000
1052/1052 [==============================] - 0s 74us/step - loss: 17.0771 - val_loss: 21.4166
Epoch 219/1000
1052/1052 [==============================] - 0s 74us/step - loss: 17.0707 - val_loss: 21.4018
Epoch 220/1000
1052/1052 [==============================] - 0s 75us/step - loss: 17.0451 - val_loss: 21.4061
Epoch 221/1000
1052/1052 [==============================] - 0s 77us/step - loss: 17.0284 - val_loss: 21.3750
Epoch 222/1000
1052/1052 [==============================] - 0s 69us/step - loss: 17.0177 - val_loss: 21.3756
Epoch 223/1000
1052/1052 [==============================] - 0s 68us/step - loss: 17.0191 - val_loss: 21.3213
Epoch 224/1000
1052/1052 [==============================] - 0s 73us/step - loss: 17.0277 - val_loss: 21.3582
Epoch 225/1000
1052/1052 [==============================] - 0s 73us/step - loss: 17.0208 - val_loss: 21.3335
Epoch 226/1000
1052/1052 [========

1052/1052 [==============================] - 0s 82us/step - loss: 15.9815 - val_loss: 19.3472
Epoch 368/1000
1052/1052 [==============================] - 0s 77us/step - loss: 15.9829 - val_loss: 19.4005
Epoch 369/1000
1052/1052 [==============================] - 0s 70us/step - loss: 15.9890 - val_loss: 19.3722
Epoch 370/1000
1052/1052 [==============================] - 0s 71us/step - loss: 15.9630 - val_loss: 19.4015
Epoch 371/1000
1052/1052 [==============================] - 0s 65us/step - loss: 15.9595 - val_loss: 19.3827
Epoch 372/1000
1052/1052 [==============================] - 0s 69us/step - loss: 15.9615 - val_loss: 19.3627
Epoch 373/1000
1052/1052 [==============================] - 0s 73us/step - loss: 15.9496 - val_loss: 19.3416
Epoch 374/1000
1052/1052 [==============================] - 0s 73us/step - loss: 15.9747 - val_loss: 19.3212
Epoch 375/1000
1052/1052 [==============================] - 0s 72us/step - loss: 15.9436 - val_loss: 19.3489
Epoch 376/1000
1052/1052 [========

1052/1052 [==============================] - 0s 75us/step - loss: 15.4383 - val_loss: 18.7374
Epoch 518/1000
1052/1052 [==============================] - 0s 66us/step - loss: 15.4279 - val_loss: 18.6796
Epoch 519/1000
1052/1052 [==============================] - 0s 76us/step - loss: 15.4407 - val_loss: 18.6863
Epoch 520/1000
1052/1052 [==============================] - 0s 75us/step - loss: 15.4280 - val_loss: 18.7064
Epoch 521/1000
1052/1052 [==============================] - 0s 70us/step - loss: 15.4659 - val_loss: 18.7418
Epoch 522/1000
1052/1052 [==============================] - 0s 71us/step - loss: 15.4271 - val_loss: 18.6630
Epoch 523/1000
1052/1052 [==============================] - 0s 74us/step - loss: 15.4376 - val_loss: 18.7030
Epoch 524/1000
1052/1052 [==============================] - 0s 72us/step - loss: 15.4168 - val_loss: 18.6639
Epoch 525/1000
1052/1052 [==============================] - 0s 72us/step - loss: 15.4185 - val_loss: 18.6714
Epoch 526/1000
1052/1052 [========

1052/1052 [==============================] - 0s 75us/step - loss: 15.2320 - val_loss: 18.4836
Epoch 593/1000
1052/1052 [==============================] - 0s 77us/step - loss: 15.2400 - val_loss: 18.5265
Epoch 594/1000
1052/1052 [==============================] - 0s 71us/step - loss: 15.2288 - val_loss: 18.4897
Epoch 595/1000
1052/1052 [==============================] - 0s 75us/step - loss: 15.2263 - val_loss: 18.4869
Epoch 596/1000
1052/1052 [==============================] - 0s 72us/step - loss: 15.2179 - val_loss: 18.5115
Epoch 597/1000
1052/1052 [==============================] - 0s 71us/step - loss: 15.2305 - val_loss: 18.4748
Epoch 598/1000
1052/1052 [==============================] - 0s 70us/step - loss: 15.2279 - val_loss: 18.4923
Epoch 599/1000
1052/1052 [==============================] - 0s 68us/step - loss: 15.2215 - val_loss: 18.5472
Epoch 600/1000
1052/1052 [==============================] - 0s 70us/step - loss: 15.2317 - val_loss: 18.4630
Epoch 601/1000
1052/1052 [========

1052/1052 [==============================] - 0s 73us/step - loss: 14.9020 - val_loss: 18.2209
Epoch 743/1000
1052/1052 [==============================] - 0s 75us/step - loss: 14.9094 - val_loss: 18.2219
Epoch 744/1000
1052/1052 [==============================] - 0s 69us/step - loss: 14.9422 - val_loss: 18.2627
Epoch 745/1000
1052/1052 [==============================] - 0s 70us/step - loss: 14.8903 - val_loss: 18.2074
Epoch 746/1000
1052/1052 [==============================] - 0s 67us/step - loss: 14.8916 - val_loss: 18.1661
Epoch 747/1000
1052/1052 [==============================] - 0s 70us/step - loss: 14.8950 - val_loss: 18.2370
Epoch 748/1000
1052/1052 [==============================] - 0s 70us/step - loss: 14.8795 - val_loss: 18.2190
Epoch 749/1000
1052/1052 [==============================] - 0s 64us/step - loss: 14.8878 - val_loss: 18.1951
Epoch 750/1000
1052/1052 [==============================] - 0s 71us/step - loss: 14.8745 - val_loss: 18.2190
Epoch 751/1000
1052/1052 [========

1052/1052 [==============================] - 0s 75us/step - loss: 14.5958 - val_loss: 18.0234
Epoch 893/1000
1052/1052 [==============================] - 0s 71us/step - loss: 14.6108 - val_loss: 18.0789
Epoch 894/1000
1052/1052 [==============================] - 0s 69us/step - loss: 14.5872 - val_loss: 18.0277
Epoch 895/1000
1052/1052 [==============================] - 0s 72us/step - loss: 14.5880 - val_loss: 18.0465
Epoch 896/1000
1052/1052 [==============================] - 0s 70us/step - loss: 14.5800 - val_loss: 18.0352
Epoch 897/1000
1052/1052 [==============================] - 0s 85us/step - loss: 14.5871 - val_loss: 18.0000
Epoch 898/1000
1052/1052 [==============================] - 0s 75us/step - loss: 14.5947 - val_loss: 18.0674
Epoch 899/1000
1052/1052 [==============================] - 0s 69us/step - loss: 14.5889 - val_loss: 18.0088
Epoch 900/1000
1052/1052 [==============================] - 0s 74us/step - loss: 14.5671 - val_loss: 18.0374
Epoch 901/1000
1052/1052 [========

[array([[-0.13033597,  0.00303281,  1.7227659 , -1.0531831 , -0.17828006],
        [-0.18282004, -0.07353149,  0.19935131, -0.00593473, -0.29323864],
        [-0.2028655 ,  0.13727102,  0.56147593, -0.31900823, -0.27328742],
        [ 0.18218863, -0.17772901, -0.11527186,  0.04287469, -0.03190209],
        [-0.11026367,  0.10199023,  0.28287208, -0.36970282, -0.10065217]],
       dtype=float32),
 array([ 1.3502218 , -1.8546903 ,  0.16378397, -1.6417633 ,  1.236223  ],
       dtype=float32),
 array([[ 0.2135845 , -0.6219347 ,  1.1291891 ,  0.6353949 , -0.443345  ,
          1.1553172 ,  1.635895  ,  1.5232306 , -0.18809892, -0.66794246],
        [ 0.3326957 ,  0.53798354, -1.0827736 , -0.7132953 ,  1.5540054 ,
         -1.242647  , -1.556107  , -0.94100356, -0.94831026,  0.08904866],
        [-0.23788857, -0.21416181, -0.6978882 , -0.30026183,  0.00361623,
         -0.5643646 , -0.76579994, -0.3459301 , -0.31992802,  0.30696556],
        [ 0.36328542,  1.1176475 , -0.3797254 , -0.869709

In [24]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_sgd.h5')

In [25]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, RMSprop, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_5 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_6 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 178us/step - l

1052/1052 [==============================] - 0s 84us/step - loss: 22.5814 - val_loss: 25.4380
Epoch 68/1000
1052/1052 [==============================] - 0s 79us/step - loss: 23.8582 - val_loss: 25.9379
Epoch 69/1000
1052/1052 [==============================] - 0s 82us/step - loss: 23.2364 - val_loss: 25.1508
Epoch 70/1000
1052/1052 [==============================] - 0s 79us/step - loss: 23.1497 - val_loss: 29.2185
Epoch 71/1000
1052/1052 [==============================] - 0s 80us/step - loss: 22.4130 - val_loss: 29.0687
Epoch 72/1000
1052/1052 [==============================] - 0s 78us/step - loss: 23.4235 - val_loss: 26.1855
Epoch 73/1000
1052/1052 [==============================] - 0s 82us/step - loss: 24.1645 - val_loss: 26.9188
Epoch 74/1000
1052/1052 [==============================] - 0s 80us/step - loss: 23.0503 - val_loss: 25.7331
Epoch 75/1000
1052/1052 [==============================] - 0s 78us/step - loss: 23.2031 - val_loss: 29.1827
Epoch 76/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 83us/step - loss: 12.7100 - val_loss: 18.7153
Epoch 218/1000
1052/1052 [==============================] - 0s 80us/step - loss: 11.9562 - val_loss: 15.7216
Epoch 219/1000
1052/1052 [==============================] - 0s 80us/step - loss: 12.1065 - val_loss: 14.3248
Epoch 220/1000
1052/1052 [==============================] - 0s 77us/step - loss: 12.1020 - val_loss: 15.1884
Epoch 221/1000
1052/1052 [==============================] - 0s 79us/step - loss: 11.9717 - val_loss: 17.3455
Epoch 222/1000
1052/1052 [==============================] - 0s 83us/step - loss: 12.3575 - val_loss: 17.2771
Epoch 223/1000
1052/1052 [==============================] - 0s 80us/step - loss: 11.8749 - val_loss: 13.1186
Epoch 224/1000
1052/1052 [==============================] - 0s 80us/step - loss: 12.2094 - val_loss: 14.2753
Epoch 225/1000
1052/1052 [==============================] - 0s 75us/step - loss: 12.5602 - val_loss: 15.2283
Epoch 226/1000
1052/1052 [========

1052/1052 [==============================] - 0s 85us/step - loss: 9.5668 - val_loss: 15.3282
Epoch 368/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.9079 - val_loss: 10.1731
Epoch 369/1000
1052/1052 [==============================] - 0s 79us/step - loss: 9.6590 - val_loss: 10.3643
Epoch 370/1000
1052/1052 [==============================] - 0s 81us/step - loss: 9.6231 - val_loss: 11.6582
Epoch 371/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.6320 - val_loss: 12.1280
Epoch 372/1000
1052/1052 [==============================] - 0s 81us/step - loss: 9.4178 - val_loss: 11.2604
Epoch 373/1000
1052/1052 [==============================] - 0s 81us/step - loss: 9.5555 - val_loss: 10.3800
Epoch 374/1000
1052/1052 [==============================] - 0s 83us/step - loss: 9.5275 - val_loss: 10.3614
Epoch 375/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.9578 - val_loss: 13.3340
Epoch 376/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 84us/step - loss: 9.4541 - val_loss: 13.7589
Epoch 444/1000
1052/1052 [==============================] - 0s 76us/step - loss: 9.8396 - val_loss: 11.3595
Epoch 445/1000
1052/1052 [==============================] - 0s 83us/step - loss: 9.7795 - val_loss: 11.6462
Epoch 446/1000
1052/1052 [==============================] - 0s 79us/step - loss: 8.8474 - val_loss: 10.4240
Epoch 447/1000
1052/1052 [==============================] - 0s 79us/step - loss: 9.2970 - val_loss: 12.0565
Epoch 448/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.1627 - val_loss: 9.9575
Epoch 449/1000
1052/1052 [==============================] - 0s 82us/step - loss: 9.0054 - val_loss: 13.9717
Epoch 450/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.2877 - val_loss: 11.5575
Epoch 451/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.8214 - val_loss: 10.4048
Epoch 452/1000
1052/1052 [==================

1052/1052 [==============================] - 0s 79us/step - loss: 8.8892 - val_loss: 11.1135
Epoch 520/1000
1052/1052 [==============================] - 0s 77us/step - loss: 9.6676 - val_loss: 12.2904
Epoch 521/1000
1052/1052 [==============================] - 0s 76us/step - loss: 9.4235 - val_loss: 10.8504
Epoch 522/1000
1052/1052 [==============================] - 0s 77us/step - loss: 9.0757 - val_loss: 13.3075
Epoch 523/1000
1052/1052 [==============================] - 0s 75us/step - loss: 9.5606 - val_loss: 10.0502
Epoch 524/1000
1052/1052 [==============================] - 0s 75us/step - loss: 9.5200 - val_loss: 10.3897
Epoch 525/1000
1052/1052 [==============================] - 0s 76us/step - loss: 9.2699 - val_loss: 11.2341
Epoch 526/1000
1052/1052 [==============================] - 0s 78us/step - loss: 9.3571 - val_loss: 10.6253
Epoch 527/1000
1052/1052 [==============================] - 0s 81us/step - loss: 8.9989 - val_loss: 14.6826
Epoch 528/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 76us/step - loss: 9.4439 - val_loss: 9.7589
Epoch 672/1000
1052/1052 [==============================] - 0s 74us/step - loss: 9.2224 - val_loss: 10.7086
Epoch 673/1000
1052/1052 [==============================] - 0s 82us/step - loss: 9.4353 - val_loss: 10.1495
Epoch 674/1000
1052/1052 [==============================] - 0s 79us/step - loss: 9.3870 - val_loss: 9.6937
Epoch 675/1000
1052/1052 [==============================] - 0s 81us/step - loss: 9.0922 - val_loss: 10.0849
Epoch 676/1000
1052/1052 [==============================] - 0s 78us/step - loss: 9.3733 - val_loss: 13.3896
Epoch 677/1000
1052/1052 [==============================] - 0s 82us/step - loss: 9.4696 - val_loss: 11.5546
Epoch 678/1000
1052/1052 [==============================] - 0s 83us/step - loss: 9.0990 - val_loss: 10.8745
Epoch 679/1000
1052/1052 [==============================] - 0s 80us/step - loss: 8.9947 - val_loss: 9.8591
Epoch 680/1000
1052/1052 [====================

1052/1052 [==============================] - 0s 82us/step - loss: 8.8875 - val_loss: 11.9768
Epoch 824/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.2874 - val_loss: 9.8136
Epoch 825/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.2799 - val_loss: 9.5913
Epoch 826/1000
1052/1052 [==============================] - 0s 78us/step - loss: 9.0596 - val_loss: 9.6589
Epoch 827/1000
1052/1052 [==============================] - 0s 75us/step - loss: 9.2857 - val_loss: 10.3445
Epoch 828/1000
1052/1052 [==============================] - 0s 78us/step - loss: 9.0509 - val_loss: 10.3620
Epoch 829/1000
1052/1052 [==============================] - 0s 78us/step - loss: 9.1552 - val_loss: 11.0458
Epoch 830/1000
1052/1052 [==============================] - 0s 81us/step - loss: 9.1378 - val_loss: 10.9479
Epoch 831/1000
1052/1052 [==============================] - 0s 78us/step - loss: 9.1000 - val_loss: 9.8757
Epoch 832/1000
1052/1052 [=====================

1052/1052 [==============================] - 0s 81us/step - loss: 9.1482 - val_loss: 10.5584
Epoch 976/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.2704 - val_loss: 9.3596
Epoch 977/1000
1052/1052 [==============================] - 0s 82us/step - loss: 9.2949 - val_loss: 9.4754
Epoch 978/1000
1052/1052 [==============================] - 0s 81us/step - loss: 9.3136 - val_loss: 10.2756
Epoch 979/1000
1052/1052 [==============================] - 0s 81us/step - loss: 9.3032 - val_loss: 11.7979
Epoch 980/1000
1052/1052 [==============================] - 0s 81us/step - loss: 9.1763 - val_loss: 9.6323
Epoch 981/1000
1052/1052 [==============================] - 0s 78us/step - loss: 8.9183 - val_loss: 11.8745
Epoch 982/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.3701 - val_loss: 11.3295
Epoch 983/1000
1052/1052 [==============================] - 0s 81us/step - loss: 9.2483 - val_loss: 15.8180
Epoch 984/1000
1052/1052 [====================

[array([[ 0.23980168, -3.0775476 ,  1.9238695 ,  4.074303  , -0.52805436],
        [ 0.26333433,  0.03217886, -0.42205986, -0.2658044 , -0.92313665],
        [ 0.39308098, -0.01686005, -0.3811762 ,  0.75786495, -1.1825671 ],
        [-0.20145257,  0.07780248, -0.08081143, -0.14767772,  0.21002544],
        [ 0.21653584, -2.5301168 ,  0.285228  ,  0.37229577, -0.02353309]],
       dtype=float32),
 array([-1.9731987, -4.580515 ,  1.2496729,  4.903427 , -0.6276174],
       dtype=float32),
 array([[-1.700869  , -2.147959  , -2.216699  , -2.0911913 , -1.9612491 ,
         -1.5509742 ,  2.0992248 , -2.1024323 ,  1.4332374 , -1.112386  ],
        [-1.0053661 , -1.7048118 , -1.6520998 , -1.6725256 , -2.0499234 ,
         -0.74319756,  1.5714961 , -1.7908423 ,  0.7960452 , -1.581173  ],
        [-1.0406294 , -0.7054909 , -0.18988483, -0.9183626 ,  0.12353275,
         -0.64403975, -0.06064613, -0.88460356,  0.82952344, -0.4889287 ],
        [ 2.1049173 ,  1.156369  ,  1.6272953 ,  1.5907923 ,  

In [26]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_rmsprop.h5')

In [27]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_7 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_8 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 581us/step - loss:

Epoch 146/1000
274/274 [==============================] - 0s 95us/step - loss: 5.9285 - val_loss: 9.7464
Epoch 147/1000
274/274 [==============================] - 0s 106us/step - loss: 6.0453 - val_loss: 9.9717
Epoch 148/1000
274/274 [==============================] - 0s 102us/step - loss: 5.9808 - val_loss: 9.6072
Epoch 149/1000
274/274 [==============================] - 0s 102us/step - loss: 5.9368 - val_loss: 9.7358
Epoch 150/1000
274/274 [==============================] - 0s 106us/step - loss: 5.8901 - val_loss: 9.6389
Epoch 151/1000
274/274 [==============================] - 0s 98us/step - loss: 5.9054 - val_loss: 9.8585
Epoch 152/1000
274/274 [==============================] - 0s 105us/step - loss: 5.9235 - val_loss: 9.6057
Epoch 153/1000
274/274 [==============================] - 0s 104us/step - loss: 5.8880 - val_loss: 9.7094
Epoch 154/1000
274/274 [==============================] - 0s 102us/step - loss: 5.8921 - val_loss: 9.8724
Epoch 155/1000
274/274 [========================

274/274 [==============================] - 0s 99us/step - loss: 5.7460 - val_loss: 10.5867
Epoch 224/1000
274/274 [==============================] - 0s 106us/step - loss: 5.9961 - val_loss: 10.5360
Epoch 225/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7454 - val_loss: 10.2242
Epoch 226/1000
274/274 [==============================] - 0s 98us/step - loss: 5.7197 - val_loss: 10.0623
Epoch 227/1000
274/274 [==============================] - 0s 98us/step - loss: 5.6296 - val_loss: 10.6349
Epoch 228/1000
274/274 [==============================] - 0s 98us/step - loss: 5.6657 - val_loss: 10.5903
Epoch 229/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6886 - val_loss: 10.0545
Epoch 230/1000
274/274 [==============================] - 0s 98us/step - loss: 5.6594 - val_loss: 10.3533
Epoch 231/1000
274/274 [==============================] - 0s 91us/step - loss: 5.5175 - val_loss: 10.3204
Epoch 232/1000
274/274 [==============================] - 0s

Epoch 300/1000
274/274 [==============================] - 0s 91us/step - loss: 5.4829 - val_loss: 10.6586
Epoch 301/1000
274/274 [==============================] - 0s 95us/step - loss: 5.4242 - val_loss: 10.3881
Epoch 302/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6511 - val_loss: 10.6812
Epoch 303/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6555 - val_loss: 10.8177
Epoch 304/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4889 - val_loss: 10.9554
Epoch 305/1000
274/274 [==============================] - 0s 102us/step - loss: 5.5073 - val_loss: 11.0437
Epoch 306/1000
274/274 [==============================] - 0s 102us/step - loss: 5.4992 - val_loss: 10.7730
Epoch 307/1000
274/274 [==============================] - 0s 102us/step - loss: 5.4845 - val_loss: 10.3521
Epoch 308/1000
274/274 [==============================] - 0s 102us/step - loss: 5.5970 - val_loss: 10.5264
Epoch 309/1000
274/274 [==================

Epoch 453/1000
274/274 [==============================] - 0s 100us/step - loss: 5.3845 - val_loss: 12.0232
Epoch 454/1000
274/274 [==============================] - 0s 93us/step - loss: 5.7061 - val_loss: 11.0984
Epoch 455/1000
274/274 [==============================] - 0s 102us/step - loss: 5.9067 - val_loss: 11.5055
Epoch 456/1000
274/274 [==============================] - 0s 102us/step - loss: 5.7650 - val_loss: 11.5303
Epoch 457/1000
274/274 [==============================] - 0s 95us/step - loss: 5.8158 - val_loss: 10.8990
Epoch 458/1000
274/274 [==============================] - 0s 98us/step - loss: 5.5119 - val_loss: 11.2075
Epoch 459/1000
274/274 [==============================] - 0s 95us/step - loss: 5.3691 - val_loss: 11.3164
Epoch 460/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3226 - val_loss: 11.3673
Epoch 461/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4815 - val_loss: 10.8674
Epoch 462/1000
274/274 [===================

...

Epoch 760/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2949 - val_loss: 11.8282
Epoch 761/1000
274/274 [==============================] - 0s 95us/step - loss: 5.1600 - val_loss: 12.0374
Epoch 762/1000
274/274 [==============================] - 0s 91us/step - loss: 5.2624 - val_loss: 12.0132
Epoch 763/1000
274/274 [==============================] - 0s 124us/step - loss: 5.1694 - val_loss: 11.6709
Epoch 764/1000
274/274 [==============================] - 0s 106us/step - loss: 5.2282 - val_loss: 11.6640
Epoch 765/1000
274/274 [==============================] - 0s 95us/step - loss: 5.2857 - val_loss: 11.7629
Epoch 766/1000
274/274 [==============================] - 0s 91us/step - loss: 5.5098 - val_loss: 12.1379
Epoch 767/1000
274/274 [==============================] - 0s 95us/step - loss: 5.3596 - val_loss: 11.7012
Epoch 768/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2821 - val_loss: 11.6081
Epoch 769/1000
274/274 [===================

Epoch 837/1000
274/274 [==============================] - 0s 106us/step - loss: 5.1509 - val_loss: 12.3988
Epoch 838/1000
274/274 [==============================] - 0s 106us/step - loss: 5.3916 - val_loss: 12.3196
Epoch 839/1000
274/274 [==============================] - 0s 109us/step - loss: 5.2459 - val_loss: 11.8300
Epoch 840/1000
274/274 [==============================] - 0s 106us/step - loss: 5.1556 - val_loss: 12.0909
Epoch 841/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3211 - val_loss: 11.5260
Epoch 842/1000
274/274 [==============================] - 0s 106us/step - loss: 5.2707 - val_loss: 11.6257
Epoch 843/1000
274/274 [==============================] - 0s 98us/step - loss: 5.5328 - val_loss: 12.2813
Epoch 844/1000
274/274 [==============================] - 0s 113us/step - loss: 5.7643 - val_loss: 13.2464
Epoch 845/1000
274/274 [==============================] - 0s 113us/step - loss: 5.3908 - val_loss: 12.3223
Epoch 846/1000
274/274 [===============

Epoch 914/1000
274/274 [==============================] - 0s 99us/step - loss: 5.5550 - val_loss: 11.8459
Epoch 915/1000
274/274 [==============================] - 0s 102us/step - loss: 5.3163 - val_loss: 11.9548
Epoch 916/1000
274/274 [==============================] - 0s 95us/step - loss: 5.1699 - val_loss: 11.8072
Epoch 917/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2136 - val_loss: 11.9626
Epoch 918/1000
274/274 [==============================] - 0s 99us/step - loss: 5.2633 - val_loss: 12.4002
Epoch 919/1000
274/274 [==============================] - 0s 95us/step - loss: 5.1932 - val_loss: 12.2517
Epoch 920/1000
274/274 [==============================] - 0s 101us/step - loss: 5.5100 - val_loss: 12.0903
Epoch 921/1000
274/274 [==============================] - 0s 101us/step - loss: 5.7015 - val_loss: 12.0285
Epoch 922/1000
274/274 [==============================] - 0s 100us/step - loss: 5.4763 - val_loss: 12.3103
Epoch 923/1000
274/274 [==================

Epoch 991/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2057 - val_loss: 11.8983
Epoch 992/1000
274/274 [==============================] - 0s 95us/step - loss: 5.1482 - val_loss: 12.4020
Epoch 993/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3537 - val_loss: 12.0096
Epoch 994/1000
274/274 [==============================] - 0s 98us/step - loss: 5.6310 - val_loss: 12.5196
Epoch 995/1000
274/274 [==============================] - 0s 95us/step - loss: 5.3094 - val_loss: 11.7051
Epoch 996/1000
274/274 [==============================] - 0s 91us/step - loss: 5.5693 - val_loss: 12.5327
Epoch 997/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3736 - val_loss: 12.4900
Epoch 998/1000
274/274 [==============================] - 0s 91us/step - loss: 5.1659 - val_loss: 12.5420
Epoch 999/1000
274/274 [==============================] - 0s 95us/step - loss: 5.1646 - val_loss: 12.0669
Epoch 1000/1000
274/274 [=====================

[array([[ 1.394878  , -1.4884644 ,  1.703876  ,  0.4099867 , -5.5041075 ],
        [-0.30577022, -0.26036423, -0.50021327, -0.74712986,  2.554801  ],
        [ 0.06273019,  1.8117826 ,  0.5501748 ,  0.2434969 , -2.0181887 ],
        [-0.6284594 , -0.25257117,  0.5892034 ,  0.478446  , -1.4118404 ],
        [ 0.2047307 ,  1.1138835 , -0.01824114, -0.14766394,  0.00887501],
        [-0.3404026 ,  0.2607666 , -0.45395944, -1.1378863 ,  1.9377538 ],
        [-0.19011907, -1.5919288 , -0.420459  , -0.019365  ,  1.8430926 ]],
       dtype=float32),
 array([ 0.8397186 , -2.3327327 , -0.01653498,  0.91879916,  0.93806285],
       dtype=float32),
 array([[-0.34428728,  0.071261  , -0.9571967 ,  0.37676704, -0.14425221,
          0.7226811 ,  0.7470566 ,  0.83972204, -0.8070323 ,  0.31378728],
        [ 0.07172816, -0.27309045,  0.06666806, -0.2625405 , -0.42119864,
         -0.46323612, -0.3162143 , -0.93956566,  0.17364994, -0.25602219],
        [ 1.5484027 , -0.71763873,  1.6585562 , -1.84733

In [28]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_adam.h5')

In [29]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, sgd, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_9 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_14 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_10 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 510us/step - loss:

274/274 [==============================] - 0s 76us/step - loss: 235.8278 - val_loss: 244.9023
Epoch 143/1000
274/274 [==============================] - 0s 84us/step - loss: 231.5743 - val_loss: 240.2877
Epoch 144/1000
274/274 [==============================] - 0s 80us/step - loss: 227.2866 - val_loss: 235.7844
Epoch 145/1000
274/274 [==============================] - 0s 80us/step - loss: 223.0368 - val_loss: 231.3022
Epoch 146/1000
274/274 [==============================] - 0s 76us/step - loss: 218.8471 - val_loss: 226.7630
Epoch 147/1000
274/274 [==============================] - 0s 80us/step - loss: 214.6314 - val_loss: 222.2301
Epoch 148/1000
274/274 [==============================] - 0s 84us/step - loss: 210.3676 - val_loss: 217.7991
Epoch 149/1000
274/274 [==============================] - 0s 80us/step - loss: 206.1957 - val_loss: 213.3619
Epoch 150/1000
274/274 [==============================] - ETA: 0s - loss: 185.038 - 0s 84us/step - loss: 202.0422 - val_loss: 208.9099
Epoch 15

Epoch 218/1000
274/274 [==============================] - 0s 84us/step - loss: 33.5688 - val_loss: 34.9321
Epoch 219/1000
274/274 [==============================] - 0s 80us/step - loss: 32.8974 - val_loss: 34.3345
Epoch 220/1000
274/274 [==============================] - 0s 84us/step - loss: 32.3067 - val_loss: 33.7411
Epoch 221/1000
274/274 [==============================] - 0s 76us/step - loss: 31.7274 - val_loss: 33.1714
Epoch 222/1000
274/274 [==============================] - 0s 84us/step - loss: 31.1606 - val_loss: 32.6408
Epoch 223/1000
274/274 [==============================] - 0s 84us/step - loss: 30.6396 - val_loss: 32.1238
Epoch 224/1000
274/274 [==============================] - 0s 80us/step - loss: 30.1435 - val_loss: 31.6179
Epoch 225/1000
274/274 [==============================] - 0s 80us/step - loss: 29.6457 - val_loss: 31.1593
Epoch 226/1000
274/274 [==============================] - 0s 80us/step - loss: 29.1845 - val_loss: 30.7156
Epoch 227/1000
274/274 [=============

274/274 [==============================] - 0s 76us/step - loss: 16.9741 - val_loss: 18.6774
Epoch 371/1000
274/274 [==============================] - 0s 80us/step - loss: 16.9556 - val_loss: 18.6533
Epoch 372/1000
274/274 [==============================] - 0s 84us/step - loss: 16.9301 - val_loss: 18.6355
Epoch 373/1000
274/274 [==============================] - 0s 84us/step - loss: 16.9093 - val_loss: 18.6107
Epoch 374/1000
274/274 [==============================] - 0s 76us/step - loss: 16.8883 - val_loss: 18.5888
Epoch 375/1000
274/274 [==============================] - 0s 76us/step - loss: 16.8650 - val_loss: 18.5653
Epoch 376/1000
274/274 [==============================] - 0s 78us/step - loss: 16.8448 - val_loss: 18.5421
Epoch 377/1000
274/274 [==============================] - 0s 76us/step - loss: 16.8241 - val_loss: 18.5127
Epoch 378/1000
274/274 [==============================] - 0s 80us/step - loss: 16.8026 - val_loss: 18.4902
Epoch 379/1000
274/274 [============================

274/274 [==============================] - 0s 84us/step - loss: 14.6794 - val_loss: 16.2577
Epoch 523/1000
274/274 [==============================] - 0s 80us/step - loss: 14.6668 - val_loss: 16.2484
Epoch 524/1000
274/274 [==============================] - 0s 79us/step - loss: 14.6564 - val_loss: 16.2356
Epoch 525/1000
274/274 [==============================] - 0s 91us/step - loss: 14.6457 - val_loss: 16.2263
Epoch 526/1000
274/274 [==============================] - 0s 84us/step - loss: 14.6346 - val_loss: 16.2122
Epoch 527/1000
274/274 [==============================] - 0s 84us/step - loss: 14.6247 - val_loss: 16.1997
Epoch 528/1000
274/274 [==============================] - 0s 87us/step - loss: 14.6125 - val_loss: 16.1900
Epoch 529/1000
274/274 [==============================] - 0s 87us/step - loss: 14.6032 - val_loss: 16.1811
Epoch 530/1000
274/274 [==============================] - 0s 89us/step - loss: 14.5909 - val_loss: 16.1723
Epoch 531/1000
274/274 [============================

274/274 [==============================] - 0s 84us/step - loss: 13.2861 - val_loss: 14.9470
Epoch 675/1000
274/274 [==============================] - 0s 87us/step - loss: 13.2779 - val_loss: 14.9367
Epoch 676/1000
274/274 [==============================] - 0s 84us/step - loss: 13.2698 - val_loss: 14.9284
Epoch 677/1000
274/274 [==============================] - 0s 84us/step - loss: 13.2630 - val_loss: 14.9228
Epoch 678/1000
274/274 [==============================] - 0s 80us/step - loss: 13.2553 - val_loss: 14.9092
Epoch 679/1000
274/274 [==============================] - 0s 80us/step - loss: 13.2475 - val_loss: 14.9008
Epoch 680/1000
274/274 [==============================] - 0s 77us/step - loss: 13.2380 - val_loss: 14.8963
Epoch 681/1000
274/274 [==============================] - 0s 84us/step - loss: 13.2306 - val_loss: 14.8918
Epoch 682/1000
274/274 [==============================] - 0s 84us/step - loss: 13.2233 - val_loss: 14.8879
Epoch 683/1000
274/274 [============================

274/274 [==============================] - 0s 84us/step - loss: 12.2062 - val_loss: 13.9926
Epoch 827/1000
274/274 [==============================] - 0s 87us/step - loss: 12.2011 - val_loss: 13.9897
Epoch 828/1000
274/274 [==============================] - 0s 78us/step - loss: 12.1950 - val_loss: 13.9814
Epoch 829/1000
274/274 [==============================] - 0s 80us/step - loss: 12.1871 - val_loss: 13.9749
Epoch 830/1000
274/274 [==============================] - 0s 84us/step - loss: 12.1805 - val_loss: 13.9708
Epoch 831/1000
274/274 [==============================] - 0s 80us/step - loss: 12.1740 - val_loss: 13.9684
Epoch 832/1000
274/274 [==============================] - 0s 82us/step - loss: 12.1687 - val_loss: 13.9624
Epoch 833/1000
274/274 [==============================] - 0s 76us/step - loss: 12.1641 - val_loss: 13.9589
Epoch 834/1000
274/274 [==============================] - 0s 80us/step - loss: 12.1547 - val_loss: 13.9491
Epoch 835/1000
274/274 [============================

274/274 [==============================] - 0s 83us/step - loss: 11.3129 - val_loss: 13.2507
Epoch 979/1000
274/274 [==============================] - 0s 84us/step - loss: 11.3076 - val_loss: 13.2473
Epoch 980/1000
274/274 [==============================] - 0s 80us/step - loss: 11.3024 - val_loss: 13.2434
Epoch 981/1000
274/274 [==============================] - 0s 87us/step - loss: 11.2969 - val_loss: 13.2387
Epoch 982/1000
274/274 [==============================] - 0s 76us/step - loss: 11.2921 - val_loss: 13.2363
Epoch 983/1000
274/274 [==============================] - 0s 84us/step - loss: 11.2858 - val_loss: 13.2335
Epoch 984/1000
274/274 [==============================] - 0s 81us/step - loss: 11.2816 - val_loss: 13.2315
Epoch 985/1000
274/274 [==============================] - 0s 80us/step - loss: 11.2770 - val_loss: 13.2293
Epoch 986/1000
274/274 [==============================] - 0s 81us/step - loss: 11.2706 - val_loss: 13.2240
Epoch 987/1000
274/274 [============================

[array([[-0.1589509 ,  0.288581  ,  0.02756506, -0.23423865,  0.08846465],
        [-0.5468112 ,  0.41990665,  0.1826548 , -0.37435243, -0.08257213],
        [ 0.30302823,  0.15068734, -0.07832705,  0.12217239, -0.5837464 ],
        [ 0.21054249, -0.12464371, -0.37689134, -0.7408029 ,  0.5025258 ],
        [ 0.01965409,  0.57485855,  0.5610765 ,  0.2516148 , -0.11407346],
        [-0.20426354,  0.12939483,  0.62788767,  0.06891876,  0.69022894],
        [-0.08856832, -0.337488  , -0.08539102,  0.37062246, -0.06784084]],
       dtype=float32),
 array([-0.9901753 , -1.0133439 ,  0.18916191, -0.10418037,  0.21094565],
       dtype=float32),
 array([[ 0.09275706,  1.1027613 , -0.44580683, -0.63820094, -1.198967  ,
          0.4233238 , -0.04828922, -1.0488257 , -0.13137946, -0.21146952],
        [ 0.18793471,  0.4564533 ,  0.23445234, -0.49052835, -1.0812689 ,
          0.6953258 ,  0.21358359, -0.3958399 ,  0.12150183,  0.28841454],
        [ 0.42533726,  0.12610047,  0.5879555 ,  0.08642

In [30]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_sgd.h5')

In [31]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, RMSprop, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_11 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_12 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 619us/step - loss:

Epoch 69/1000
274/274 [==============================] - 0s 84us/step - loss: 8.4753 - val_loss: 11.5714
Epoch 70/1000
274/274 [==============================] - 0s 91us/step - loss: 8.5609 - val_loss: 13.3235
Epoch 71/1000
274/274 [==============================] - 0s 87us/step - loss: 7.8597 - val_loss: 11.9162
Epoch 72/1000
274/274 [==============================] - 0s 95us/step - loss: 8.2388 - val_loss: 11.4134
Epoch 73/1000
274/274 [==============================] - 0s 87us/step - loss: 8.2693 - val_loss: 11.1458
Epoch 74/1000
274/274 [==============================] - 0s 87us/step - loss: 8.9107 - val_loss: 11.6571
Epoch 75/1000
274/274 [==============================] - 0s 87us/step - loss: 8.1438 - val_loss: 11.3947
Epoch 76/1000
274/274 [==============================] - 0s 80us/step - loss: 8.1884 - val_loss: 12.7839
Epoch 77/1000
274/274 [==============================] - 0s 80us/step - loss: 8.0554 - val_loss: 13.9641
Epoch 78/1000
274/274 [==============================] 

274/274 [==============================] - 0s 57us/step - loss: 7.2748 - val_loss: 12.2731
Epoch 224/1000
274/274 [==============================] - 0s 57us/step - loss: 7.6683 - val_loss: 11.9225
Epoch 225/1000
274/274 [==============================] - 0s 57us/step - loss: 7.1493 - val_loss: 11.7583
Epoch 226/1000
274/274 [==============================] - 0s 57us/step - loss: 7.5349 - val_loss: 10.9051
Epoch 227/1000
274/274 [==============================] - 0s 114us/step - loss: 8.0057 - val_loss: 11.4342
Epoch 228/1000
274/274 [==============================] - 0s 57us/step - loss: 7.4862 - val_loss: 11.6871
Epoch 229/1000
274/274 [==============================] - 0s 114us/step - loss: 7.3259 - val_loss: 12.3830
Epoch 230/1000
274/274 [==============================] - 0s 57us/step - loss: 7.3927 - val_loss: 12.5251
Epoch 231/1000
274/274 [==============================] - 0s 114us/step - loss: 7.8161 - val_loss: 11.8548
Epoch 232/1000
274/274 [==============================] - 

274/274 [==============================] - 0s 139us/step - loss: 7.3350 - val_loss: 12.1118
Epoch 301/1000
274/274 [==============================] - 0s 114us/step - loss: 7.2129 - val_loss: 11.3489
Epoch 302/1000
274/274 [==============================] - 0s 88us/step - loss: 7.3799 - val_loss: 11.6308
Epoch 303/1000
274/274 [==============================] - 0s 57us/step - loss: 7.3608 - val_loss: 11.6068
Epoch 304/1000
274/274 [==============================] - 0s 114us/step - loss: 6.9014 - val_loss: 12.3747
Epoch 305/1000
274/274 [==============================] - 0s 57us/step - loss: 7.1971 - val_loss: 12.3374
Epoch 306/1000
274/274 [==============================] - 0s 57us/step - loss: 7.2835 - val_loss: 10.9748
Epoch 307/1000
274/274 [==============================] - 0s 57us/step - loss: 7.4962 - val_loss: 12.1434
Epoch 308/1000
274/274 [==============================] - 0s 57us/step - loss: 7.3840 - val_loss: 12.4218
Epoch 309/1000
274/274 [==============================] - 

274/274 [==============================] - 0s 114us/step - loss: 7.0440 - val_loss: 11.3737
Epoch 378/1000
274/274 [==============================] - 0s 57us/step - loss: 7.2470 - val_loss: 12.1223
Epoch 379/1000
274/274 [==============================] - 0s 57us/step - loss: 6.6963 - val_loss: 15.0638
Epoch 380/1000
274/274 [==============================] - 0s 114us/step - loss: 7.0510 - val_loss: 10.5599
Epoch 381/1000
274/274 [==============================] - 0s 57us/step - loss: 7.2944 - val_loss: 12.1963
Epoch 382/1000
274/274 [==============================] - 0s 57us/step - loss: 6.6463 - val_loss: 14.0671
Epoch 383/1000
274/274 [==============================] - 0s 57us/step - loss: 7.3205 - val_loss: 11.3994
Epoch 384/1000
274/274 [==============================] - 0s 57us/step - loss: 7.1374 - val_loss: 12.7164
Epoch 385/1000
274/274 [==============================] - 0s 57us/step - loss: 6.8123 - val_loss: 11.1717
Epoch 386/1000
274/274 [==============================] - 0

...

Epoch 684/1000
274/274 [==============================] - 0s 114us/step - loss: 6.3537 - val_loss: 10.9512
Epoch 685/1000
274/274 [==============================] - 0s 114us/step - loss: 6.1458 - val_loss: 10.6181
Epoch 686/1000
274/274 [==============================] - 0s 57us/step - loss: 6.3981 - val_loss: 11.6765
Epoch 687/1000
274/274 [==============================] - 0s 57us/step - loss: 5.9668 - val_loss: 11.4846
Epoch 688/1000
274/274 [==============================] - 0s 114us/step - loss: 6.1673 - val_loss: 11.0289
Epoch 689/1000
274/274 [==============================] - 0s 57us/step - loss: 6.0209 - val_loss: 12.7273
Epoch 690/1000
274/274 [==============================] - 0s 57us/step - loss: 6.4373 - val_loss: 11.9094
Epoch 691/1000
274/274 [==============================] - 0s 114us/step - loss: 6.2299 - val_loss: 10.7475
Epoch 692/1000
274/274 [==============================] - 0s 57us/step - loss: 6.0303 - val_loss: 11.3679
Epoch 693/1000
274/274 [==================

Epoch 761/1000
274/274 [==============================] - 0s 57us/step - loss: 5.6431 - val_loss: 12.7390
Epoch 762/1000
274/274 [==============================] - 0s 114us/step - loss: 6.2416 - val_loss: 11.7602
Epoch 763/1000
274/274 [==============================] - 0s 114us/step - loss: 6.3325 - val_loss: 11.4936
Epoch 764/1000
274/274 [==============================] - 0s 57us/step - loss: 6.1124 - val_loss: 10.9814
Epoch 765/1000
274/274 [==============================] - 0s 142us/step - loss: 5.6907 - val_loss: 11.4795
Epoch 766/1000
274/274 [==============================] - 0s 66us/step - loss: 6.0762 - val_loss: 11.0601
Epoch 767/1000
274/274 [==============================] - 0s 114us/step - loss: 6.0098 - val_loss: 10.3093
Epoch 768/1000
274/274 [==============================] - 0s 57us/step - loss: 5.5897 - val_loss: 11.7634
Epoch 769/1000
274/274 [==============================] - 0s 114us/step - loss: 6.1063 - val_loss: 10.3914
Epoch 770/1000
274/274 [=================

274/274 [==============================] - 0s 114us/step - loss: 5.9987 - val_loss: 11.9360
Epoch 839/1000
274/274 [==============================] - 0s 57us/step - loss: 5.6732 - val_loss: 11.5611
Epoch 840/1000
274/274 [==============================] - 0s 114us/step - loss: 5.6476 - val_loss: 10.9355
Epoch 841/1000
274/274 [==============================] - 0s 57us/step - loss: 5.8753 - val_loss: 10.3356
Epoch 842/1000
274/274 [==============================] - 0s 57us/step - loss: 6.0759 - val_loss: 10.8213
Epoch 843/1000
274/274 [==============================] - 0s 114us/step - loss: 5.4161 - val_loss: 11.5302
Epoch 844/1000
274/274 [==============================] - 0s 57us/step - loss: 5.6748 - val_loss: 12.2719
Epoch 845/1000
274/274 [==============================] - 0s 114us/step - loss: 5.6125 - val_loss: 10.6945
Epoch 846/1000
274/274 [==============================] - 0s 57us/step - loss: 6.2262 - val_loss: 11.0961
Epoch 847/1000
274/274 [==============================] -

274/274 [==============================] - 0s 57us/step - loss: 5.4415 - val_loss: 10.3482
Epoch 992/1000
274/274 [==============================] - 0s 114us/step - loss: 5.7697 - val_loss: 11.2350
Epoch 993/1000
274/274 [==============================] - 0s 114us/step - loss: 5.6045 - val_loss: 10.0579
Epoch 994/1000
274/274 [==============================] - 0s 57us/step - loss: 5.5707 - val_loss: 10.8962
Epoch 995/1000
274/274 [==============================] - 0s 114us/step - loss: 5.5207 - val_loss: 10.5212
Epoch 996/1000
274/274 [==============================] - 0s 57us/step - loss: 5.5977 - val_loss: 11.6458
Epoch 997/1000
274/274 [==============================] - 0s 114us/step - loss: 5.6905 - val_loss: 10.4350
Epoch 998/1000
274/274 [==============================] - 0s 57us/step - loss: 5.7608 - val_loss: 11.0640
Epoch 999/1000
274/274 [==============================] - 0s 114us/step - loss: 5.4005 - val_loss: 10.2729
Epoch 1000/1000
274/274 [==============================]

[array([[-1.1746191 ,  2.4293883 , -6.91856   ,  1.9678552 , -2.2026954 ],
        [ 0.47316688, -3.016624  ,  3.4685297 ,  0.03263656,  0.16508159],
        [ 0.11575423,  0.5445551 , -2.0351822 , -0.6714933 ,  0.2927511 ],
        [ 0.46256348,  2.946144  , -2.714637  ,  0.3525459 ,  0.62859106],
        [ 0.5747    , -1.9116157 , -0.37213433, -0.12003113, -0.2591557 ],
        [-0.22660203, -0.89768785,  0.96917105,  1.1227708 ,  0.30428106],
        [-1.2314231 , -0.64644545,  0.07762497, -0.16471599,  0.11163276]],
       dtype=float32),
 array([-1.1737967, -2.9409952, -1.9340348,  1.4686694, -0.9354909],
       dtype=float32),
 array([[ 0.0528037 , -0.3853231 , -0.08422575,  0.5677087 , -0.07431588,
          0.574651  ,  1.010017  ,  0.6747852 , -0.65608954, -0.4426655 ],
        [-0.03173079,  0.08912551, -0.26017058,  1.0346096 , -0.5392855 ,
          0.90504646,  0.26732296,  0.25705466, -0.10923968, -0.7899628 ],
        [-0.9667876 ,  1.2490555 ,  0.654677  , -1.235496  , 

In [32]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_rmsprop.h5')

In [33]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_13 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_20 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_14 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 1ms/step - loss: 0

143/143 [==============================] - 0s 109us/step - loss: 0.0058 - val_loss: 0.0051
Epoch 70/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0060 - val_loss: 0.0040
Epoch 71/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0066 - val_loss: 0.0055
Epoch 72/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0063 - val_loss: 0.0041
Epoch 73/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0070 - val_loss: 0.0058
Epoch 74/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0067 - val_loss: 0.0045
Epoch 75/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0073 - val_loss: 0.0064
Epoch 76/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0073 - val_loss: 0.0041
Epoch 77/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0071 - val_loss: 0.0057
Epoch 78/1000
143/143 [==============================] - 0s 109us/ste

143/143 [==============================] - 0s 109us/step - loss: 0.0045 - val_loss: 0.0045
Epoch 224/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0049 - val_loss: 0.0035
Epoch 225/1000
143/143 [==============================] - 0s 50us/step - loss: 0.0054 - val_loss: 0.0065
Epoch 226/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0057 - val_loss: 0.0033
Epoch 227/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0043 - val_loss: 0.0050
Epoch 228/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0044 - val_loss: 0.0038
Epoch 229/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0044 - val_loss: 0.0039
Epoch 230/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0042 - val_loss: 0.0033
Epoch 231/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0043 - val_loss: 0.0046
Epoch 232/1000
143/143 [==============================] - 0s 1

143/143 [==============================] - 0s 109us/step - loss: 0.0033 - val_loss: 0.0031
Epoch 378/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0031 - val_loss: 0.0027
Epoch 379/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0032 - val_loss: 0.0041
Epoch 380/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0033 - val_loss: 0.0028
Epoch 381/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0035 - val_loss: 0.0036
Epoch 382/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0034 - val_loss: 0.0035
Epoch 383/1000
143/143 [==============================] - 0s 0us/step - loss: 0.0032 - val_loss: 0.0028
Epoch 384/1000
143/143 [==============================] - 0s 0us/step - loss: 0.0034 - val_loss: 0.0047
Epoch 385/1000
143/143 [==============================] - 0s 0us/step - loss: 0.0035 - val_loss: 0.0026
Epoch 386/1000
143/143 [==============================] - 0s 109us/

143/143 [==============================] - 0s 109us/step - loss: 0.0028 - val_loss: 0.0032
Epoch 532/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0027 - val_loss: 0.0040
Epoch 533/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0027 - val_loss: 0.0031
Epoch 534/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0028 - val_loss: 0.0035
Epoch 535/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0029 - val_loss: 0.0048
Epoch 536/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0029 - val_loss: 0.0032
Epoch 537/1000
143/143 [==============================] - 0s 170us/step - loss: 0.0024 - val_loss: 0.0037
Epoch 538/1000
143/143 [==============================] - 0s 15us/step - loss: 0.0026 - val_loss: 0.0031
Epoch 539/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0024 - val_loss: 0.0033
Epoch 540/1000
143/143 [==============================] - 0s 1

143/143 [==============================] - 0s 109us/step - loss: 0.0022 - val_loss: 0.0029
Epoch 686/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0022 - val_loss: 0.0034
Epoch 687/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0022 - val_loss: 0.0048
Epoch 688/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0027 - val_loss: 0.0030
Epoch 689/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0024 - val_loss: 0.0032
Epoch 690/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0025 - val_loss: 0.0045
Epoch 691/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0023 - val_loss: 0.0033
Epoch 692/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0024 - val_loss: 0.0032
Epoch 693/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0023 - val_loss: 0.0056
Epoch 694/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 109us/step - loss: 0.0018 - val_loss: 0.0033
Epoch 840/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0021 - val_loss: 0.0042
Epoch 841/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0017 - val_loss: 0.0029
Epoch 842/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0019 - val_loss: 0.0034
Epoch 843/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0018 - val_loss: 0.0043
Epoch 844/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0019 - val_loss: 0.0033
Epoch 845/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0017 - val_loss: 0.0038
Epoch 846/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0017 - val_loss: 0.0030
Epoch 847/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0019 - val_loss: 0.0044
Epoch 848/1000
143/143 [==============================] - ETA

143/143 [==============================] - 0s 109us/step - loss: 0.0017 - val_loss: 0.0040
Epoch 994/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0015 - val_loss: 0.0042
Epoch 995/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0018 - val_loss: 0.0028
Epoch 996/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0017 - val_loss: 0.0035
Epoch 997/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0017 - val_loss: 0.0037
Epoch 998/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0015 - val_loss: 0.0037
Epoch 999/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0016 - val_loss: 0.0040
Epoch 1000/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0016 - val_loss: 0.0031
0.008868318982422352


[array([[-2.29680791e-01, -3.25725108e-01, -1.53504517e-02,
          4.32914793e-01, -1.04041183e+00],
        [-2.23307967e-01,  1.01031056e-02, -1.22327246e-01,
         -6.72626436e-01,  2.06743762e-01],
        [ 7.41706133e-01, -3.15109611e-01,  3.25972401e-02,
          1.25615641e-01,  1.31149590e+00],
        [-2.28908216e-03, -7.10812807e-01,  8.14740360e-02,
         -3.08187038e-01,  3.74538630e-01],
        [-1.41407236e-01, -2.51665562e-01, -1.57846481e-01,
         -2.61490166e-01,  2.41048113e-01],
        [-3.31043333e-01,  1.35839563e-02, -1.02597189e+00,
         -1.74393207e-01, -8.68754923e-01],
        [-5.65763056e-01, -5.84106743e-02, -6.97795212e-01,
         -7.55099118e-01,  1.95976600e-01],
        [-3.39076430e-01,  1.83794245e-01,  2.65133917e-01,
         -3.10411751e-01, -6.68103620e-02],
        [ 8.16033334e-02, -4.71576542e-01, -7.59522676e-01,
         -4.61577848e-02,  9.88743082e-02],
        [-1.10499966e+00, -2.02303991e-01,  8.35567951e-01,
    

In [34]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_adam.h5')

In [35]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, sgd, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_15 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_16 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 1ms/step - loss: 0

143/143 [==============================] - 0s 109us/step - loss: 0.7532 - val_loss: 0.7706
Epoch 70/1000
143/143 [==============================] - 0s 109us/step - loss: 0.7502 - val_loss: 0.7675
Epoch 71/1000
143/143 [==============================] - 0s 109us/step - loss: 0.7471 - val_loss: 0.7645
Epoch 72/1000
143/143 [==============================] - 0s 109us/step - loss: 0.7442 - val_loss: 0.7615
Epoch 73/1000
143/143 [==============================] - 0s 109us/step - loss: 0.7412 - val_loss: 0.7585
Epoch 74/1000
143/143 [==============================] - 0s 109us/step - loss: 0.7382 - val_loss: 0.7555
Epoch 75/1000
143/143 [==============================] - 0s 109us/step - loss: 0.7353 - val_loss: 0.7526
Epoch 76/1000
143/143 [==============================] - 0s 122us/step - loss: 0.7323 - val_loss: 0.7496
Epoch 77/1000
143/143 [==============================] - 0s 91us/step - loss: 0.7294 - val_loss: 0.7466
Epoch 78/1000
143/143 [==============================] - 0s 93us/step 

Epoch 147/1000
143/143 [==============================] - 0s 109us/step - loss: 0.5558 - val_loss: 0.5718
Epoch 148/1000
143/143 [==============================] - 0s 109us/step - loss: 0.5537 - val_loss: 0.5697
Epoch 149/1000
143/143 [==============================] - 0s 109us/step - loss: 0.5517 - val_loss: 0.5676
Epoch 150/1000
143/143 [==============================] - 0s 109us/step - loss: 0.5496 - val_loss: 0.5655
Epoch 151/1000
143/143 [==============================] - 0s 109us/step - loss: 0.5475 - val_loss: 0.5634
Epoch 152/1000
143/143 [==============================] - 0s 0us/step - loss: 0.5454 - val_loss: 0.5613
Epoch 153/1000
143/143 [==============================] - 0s 0us/step - loss: 0.5434 - val_loss: 0.5593
Epoch 154/1000
143/143 [==============================] - 0s 109us/step - loss: 0.5413 - val_loss: 0.5572
Epoch 155/1000
143/143 [==============================] - 0s 109us/step - loss: 0.5393 - val_loss: 0.5551
Epoch 156/1000
143/143 [==========================

143/143 [==============================] - 0s 0us/step - loss: 0.3234 - val_loss: 0.3367
Epoch 302/1000
143/143 [==============================] - 0s 219us/step - loss: 0.3224 - val_loss: 0.3357
Epoch 303/1000
143/143 [==============================] - 0s 109us/step - loss: 0.3213 - val_loss: 0.3346
Epoch 304/1000
143/143 [==============================] - 0s 109us/step - loss: 0.3203 - val_loss: 0.3336
Epoch 305/1000
143/143 [==============================] - 0s 0us/step - loss: 0.3192 - val_loss: 0.3325
Epoch 306/1000
143/143 [==============================] - 0s 0us/step - loss: 0.3182 - val_loss: 0.3315
Epoch 307/1000
143/143 [==============================] - 0s 109us/step - loss: 0.3172 - val_loss: 0.3304
Epoch 308/1000
143/143 [==============================] - 0s 109us/step - loss: 0.3161 - val_loss: 0.3293
Epoch 309/1000
143/143 [==============================] - 0s 109us/step - loss: 0.3151 - val_loss: 0.3283
Epoch 310/1000
143/143 [==============================] - 0s 109us/

143/143 [==============================] - 0s 109us/step - loss: 0.2033 - val_loss: 0.2149
Epoch 456/1000
143/143 [==============================] - 0s 109us/step - loss: 0.2027 - val_loss: 0.2143
Epoch 457/1000
143/143 [==============================] - 0s 109us/step - loss: 0.2022 - val_loss: 0.2137
Epoch 458/1000
143/143 [==============================] - 0s 109us/step - loss: 0.2016 - val_loss: 0.2132
Epoch 459/1000
143/143 [==============================] - 0s 109us/step - loss: 0.2010 - val_loss: 0.2126
Epoch 460/1000
143/143 [==============================] - 0s 0us/step - loss: 0.2005 - val_loss: 0.2121
Epoch 461/1000
143/143 [==============================] - 0s 109us/step - loss: 0.1999 - val_loss: 0.2115
Epoch 462/1000
143/143 [==============================] - 0s 109us/step - loss: 0.1994 - val_loss: 0.2109
Epoch 463/1000
143/143 [==============================] - 0s 109us/step - loss: 0.1988 - val_loss: 0.2104
Epoch 464/1000
143/143 [==============================] - 0s 10

143/143 [==============================] - 0s 109us/step - loss: 0.1382 - val_loss: 0.1488
Epoch 610/1000
143/143 [==============================] - 0s 109us/step - loss: 0.1379 - val_loss: 0.1485
Epoch 611/1000
143/143 [==============================] - 0s 109us/step - loss: 0.1376 - val_loss: 0.1482
Epoch 612/1000
143/143 [==============================] - 0s 109us/step - loss: 0.1373 - val_loss: 0.1479
Epoch 613/1000
143/143 [==============================] - 0s 109us/step - loss: 0.1370 - val_loss: 0.1476
Epoch 614/1000
143/143 [==============================] - 0s 109us/step - loss: 0.1367 - val_loss: 0.1473
Epoch 615/1000
143/143 [==============================] - 0s 109us/step - loss: 0.1363 - val_loss: 0.1470
Epoch 616/1000
143/143 [==============================] - 0s 109us/step - loss: 0.1360 - val_loss: 0.1467
Epoch 617/1000
143/143 [==============================] - 0s 109us/step - loss: 0.1357 - val_loss: 0.1463
Epoch 618/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 109us/step - loss: 0.1017 - val_loss: 0.1118
Epoch 764/1000
143/143 [==============================] - 0s 76us/step - loss: 0.1015 - val_loss: 0.1116
Epoch 765/1000
143/143 [==============================] - 0s 80us/step - loss: 0.1014 - val_loss: 0.1115
Epoch 766/1000
143/143 [==============================] - 0s 109us/step - loss: 0.1012 - val_loss: 0.1113
Epoch 767/1000
143/143 [==============================] - 0s 109us/step - loss: 0.1010 - val_loss: 0.1111
Epoch 768/1000
143/143 [==============================] - 0s 0us/step - loss: 0.1008 - val_loss: 0.1109
Epoch 769/1000
143/143 [==============================] - 0s 109us/step - loss: 0.1007 - val_loss: 0.1108
Epoch 770/1000
143/143 [==============================] - 0s 109us/step - loss: 0.1005 - val_loss: 0.1106
Epoch 771/1000
143/143 [==============================] - 0s 109us/step - loss: 0.1003 - val_loss: 0.1104
Epoch 772/1000
143/143 [==============================] - 0s 109u

143/143 [==============================] - 0s 109us/step - loss: 0.0807 - val_loss: 0.0905
Epoch 918/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0806 - val_loss: 0.0904
Epoch 919/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0805 - val_loss: 0.0903
Epoch 920/1000
143/143 [==============================] - 0s 0us/step - loss: 0.0804 - val_loss: 0.0902
Epoch 921/1000
143/143 [==============================] - 0s 0us/step - loss: 0.0803 - val_loss: 0.0901
Epoch 922/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0802 - val_loss: 0.0900
Epoch 923/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0801 - val_loss: 0.0899
Epoch 924/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0800 - val_loss: 0.0898
Epoch 925/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0799 - val_loss: 0.0897
Epoch 926/1000
143/143 [==============================] - 0s 109u

[array([[-2.61135995e-01,  1.43971264e-01,  2.96889991e-01,
          3.98186326e-01, -4.12563205e-01],
        [ 1.58642039e-01,  1.32152587e-01,  3.66374373e-01,
          3.50283593e-01, -3.07209879e-01],
        [ 3.59283030e-01,  1.79995671e-01, -6.67479187e-02,
         -5.31876534e-02, -2.38777623e-02],
        [-3.34320188e-01,  2.12319285e-01,  3.15750659e-01,
         -2.26555735e-01, -4.47914004e-01],
        [-1.38410509e-01, -3.82760257e-01, -1.56087071e-01,
         -2.97018379e-01, -1.18955620e-01],
        [-4.63127106e-01,  1.68042153e-01,  2.30370909e-01,
         -1.27137169e-01, -3.57784539e-01],
        [-1.81369245e-01, -3.96997690e-01,  4.38129485e-01,
          4.78234366e-02, -3.60565186e-01],
        [-4.39341038e-01, -2.10335538e-01, -5.30892536e-02,
          6.25511110e-02, -1.49749845e-01],
        [ 4.08220440e-01,  5.71146309e-02, -4.17859524e-01,
          3.18556204e-02, -3.50507915e-01],
        [-9.63277444e-02, -4.47599500e-01, -6.11512922e-02,
    

In [36]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_sgd.h5')

In [37]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, RMSprop, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_17 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_26 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_18 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 1ms/step - loss: 0

Epoch 147/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0081 - val_loss: 0.0055
Epoch 148/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0077 - val_loss: 0.0104
Epoch 149/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0100 - val_loss: 0.0048
Epoch 150/1000
143/143 [==============================] - 0s 0us/step - loss: 0.0070 - val_loss: 0.0059
Epoch 151/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0082 - val_loss: 0.0077
Epoch 152/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0076 - val_loss: 0.0051
Epoch 153/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0095 - val_loss: 0.0068
Epoch 154/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0077 - val_loss: 0.0052
Epoch 155/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0082 - val_loss: 0.0049
Epoch 156/1000
143/143 [========================

143/143 [==============================] - 0s 109us/step - loss: 0.0046 - val_loss: 0.0044
Epoch 302/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0064 - val_loss: 0.0041
Epoch 303/1000
143/143 [==============================] - 0s 0us/step - loss: 0.0043 - val_loss: 0.0057
Epoch 304/1000
143/143 [==============================] - 0s 0us/step - loss: 0.0044 - val_loss: 0.0087
Epoch 305/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0066 - val_loss: 0.0041
Epoch 306/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0047 - val_loss: 0.0041
Epoch 307/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0045 - val_loss: 0.0065
Epoch 308/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0057 - val_loss: 0.0042
Epoch 309/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0072 - val_loss: 0.0070
Epoch 310/1000
143/143 [==============================] - 0s 109u

143/143 [==============================] - 0s 109us/step - loss: 0.0046 - val_loss: 0.0048
Epoch 456/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0034 - val_loss: 0.0064
Epoch 457/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0054 - val_loss: 0.0073
Epoch 458/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0034 - val_loss: 0.0066
Epoch 459/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0061 - val_loss: 0.0048
Epoch 460/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0031 - val_loss: 0.0048
Epoch 461/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0032 - val_loss: 0.0077
Epoch 462/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0059 - val_loss: 0.0060
Epoch 463/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0034 - val_loss: 0.0050
Epoch 464/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 109us/step - loss: 0.0035 - val_loss: 0.0053
Epoch 610/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0028 - val_loss: 0.0078
Epoch 611/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0025 - val_loss: 0.0052
Epoch 612/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0026 - val_loss: 0.0068
Epoch 613/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0028 - val_loss: 0.0090
Epoch 614/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0054 - val_loss: 0.0050
Epoch 615/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0025 - val_loss: 0.0046
Epoch 616/1000
143/143 [==============================] - 0s 0us/step - loss: 0.0025 - val_loss: 0.0050
Epoch 617/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0031 - val_loss: 0.0046
Epoch 618/1000
143/143 [==============================] - 0s 10

143/143 [==============================] - 0s 109us/step - loss: 0.0022 - val_loss: 0.0053
Epoch 764/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0024 - val_loss: 0.0044
Epoch 765/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0040 - val_loss: 0.0068
Epoch 766/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0030 - val_loss: 0.0042
Epoch 767/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0025 - val_loss: 0.0043
Epoch 768/1000
143/143 [==============================] - 0s 0us/step - loss: 0.0024 - val_loss: 0.0070
Epoch 769/1000
143/143 [==============================] - 0s 0us/step - loss: 0.0026 - val_loss: 0.0044
Epoch 770/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0029 - val_loss: 0.0047
Epoch 771/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0030 - val_loss: 0.0041
Epoch 772/1000
143/143 [==============================] - 0s 109u

143/143 [==============================] - 0s 109us/step - loss: 0.0024 - val_loss: 0.0051
Epoch 918/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0027 - val_loss: 0.0044
Epoch 919/1000
143/143 [==============================] - 0s 0us/step - loss: 0.0023 - val_loss: 0.0050
Epoch 920/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0019 - val_loss: 0.0043
Epoch 921/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0017 - val_loss: 0.0046
Epoch 922/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0035 - val_loss: 0.0054
Epoch 923/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0025 - val_loss: 0.0055
Epoch 924/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0021 - val_loss: 0.0042
Epoch 925/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0022 - val_loss: 0.0067
Epoch 926/1000
143/143 [==============================] - 0s 0u

[array([[-1.11332440e+00,  7.49457836e-01,  1.84156954e-01,
          5.71338177e-01,  1.41287923e+00],
        [ 1.54389286e+00, -3.43615770e-01,  2.35196188e-01,
          3.99971455e-01,  6.83304906e-01],
        [ 1.36241496e+00, -3.80690098e-01, -9.54309944e-05,
         -3.98674905e-01,  2.09895223e-01],
        [ 1.57396924e+00,  2.55836964e-01,  6.60162121e-02,
          4.92314696e-01,  5.97380972e+00],
        [ 6.59281492e-01, -2.26774111e-01,  2.48968929e-01,
         -4.63987917e-01,  1.63887095e+00],
        [-1.40971923e+00, -1.99838787e-01,  3.30479026e-01,
         -1.09866428e+00,  2.04394147e-01],
        [ 2.16032580e-01,  3.27382952e-01,  8.79763663e-02,
          3.98458131e-02,  5.73907316e-01],
        [-1.82221178e-02,  3.34991604e-01,  1.43153623e-01,
         -1.50669083e-01,  2.55337656e-02],
        [-4.49178278e-01, -1.15429389e+00, -5.40514942e-04,
         -9.50398803e-01,  1.24408293e+00],
        [ 1.67693142e-02,  4.34514314e-01, -1.77944183e-01,
    

In [38]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_rmsprop.h5')

In [39]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_19 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_29 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_20 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 6ms/step - loss: 36.484

Epoch 72/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0129 - val_loss: 0.0060
Epoch 73/1000
32/32 [==============================] - 0s 488us/step - loss: 0.0136 - val_loss: 0.0070
Epoch 74/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0146 - val_loss: 0.0077
Epoch 75/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0154 - val_loss: 0.0078
Epoch 76/1000
32/32 [==============================] - 0s 488us/step - loss: 0.0158 - val_loss: 0.0075
Epoch 77/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0155 - val_loss: 0.0068
Epoch 78/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0148 - val_loss: 0.0060
Epoch 79/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0138 - val_loss: 0.0052
Epoch 80/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0128 - val_loss: 0.0046
Epoch 81/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0119 - val

Epoch 153/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0034 - val_loss: 0.0010
Epoch 154/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0033 - val_loss: 0.0010
Epoch 155/1000
32/32 [==============================] - 0s 488us/step - loss: 0.0033 - val_loss: 9.9795e-04
Epoch 156/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0032 - val_loss: 9.8053e-04
Epoch 157/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0032 - val_loss: 9.6233e-04
Epoch 158/1000
32/32 [==============================] - 0s 488us/step - loss: 0.0031 - val_loss: 9.4376e-04
Epoch 159/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0031 - val_loss: 9.2524e-04
Epoch 160/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0030 - val_loss: 9.0708e-04
Epoch 161/1000
32/32 [==============================] - 0s 488us/step - loss: 0.0030 - val_loss: 8.8956e-04
Epoch 162/1000
32/32 [==========================

Epoch 230/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0014 - val_loss: 3.2598e-04
Epoch 231/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0014 - val_loss: 3.2207e-04
Epoch 232/1000
32/32 [==============================] - 0s 487us/step - loss: 0.0014 - val_loss: 3.1820e-04
Epoch 233/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0014 - val_loss: 3.1438e-04
Epoch 234/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0014 - val_loss: 3.1062e-04
Epoch 235/1000
32/32 [==============================] - 0s 488us/step - loss: 0.0014 - val_loss: 3.0690e-04
Epoch 236/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0014 - val_loss: 3.0325e-04
Epoch 237/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0014 - val_loss: 2.9965e-04
Epoch 238/1000
32/32 [==============================] - 0s 488us/step - loss: 0.0014 - val_loss: 2.9610e-04
Epoch 239/1000
32/32 [==================

Epoch 307/1000
32/32 [==============================] - 0s 0us/step - loss: 9.5435e-04 - val_loss: 1.6125e-04
Epoch 308/1000
32/32 [==============================] - 0s 488us/step - loss: 9.5068e-04 - val_loss: 1.6033e-04
Epoch 309/1000
32/32 [==============================] - 0s 0us/step - loss: 9.4704e-04 - val_loss: 1.5943e-04
Epoch 310/1000
32/32 [==============================] - 0s 0us/step - loss: 9.4344e-04 - val_loss: 1.5855e-04
Epoch 311/1000
32/32 [==============================] - 0s 0us/step - loss: 9.3987e-04 - val_loss: 1.5769e-04
Epoch 312/1000
32/32 [==============================] - 0s 0us/step - loss: 9.3633e-04 - val_loss: 1.5684e-04
Epoch 313/1000
32/32 [==============================] - 0s 0us/step - loss: 9.3281e-04 - val_loss: 1.5601e-04
Epoch 314/1000
32/32 [==============================] - 0s 0us/step - loss: 9.2934e-04 - val_loss: 1.5519e-04
Epoch 315/1000
32/32 [==============================] - 0s 488us/step - loss: 9.2589e-04 - val_loss: 1.5439e-04
Epoch 

...

Epoch 756/1000
32/32 [==============================] - 0s 0us/step - loss: 2.9400e-04 - val_loss: 6.8999e-05
Epoch 757/1000
32/32 [==============================] - 0s 0us/step - loss: 2.9334e-04 - val_loss: 6.8937e-05
Epoch 758/1000
32/32 [==============================] - 0s 0us/step - loss: 2.9270e-04 - val_loss: 6.8860e-05
Epoch 759/1000
32/32 [==============================] - 0s 0us/step - loss: 2.9205e-04 - val_loss: 6.8791e-05
Epoch 760/1000
32/32 [==============================] - 0s 127us/step - loss: 2.9140e-04 - val_loss: 6.8728e-05
Epoch 761/1000
32/32 [==============================] - 0s 164us/step - loss: 2.9076e-04 - val_loss: 6.8655e-05
Epoch 762/1000
32/32 [==============================] - 0s 62us/step - loss: 2.9011e-04 - val_loss: 6.8586e-05
Epoch 763/1000
32/32 [==============================] - 0s 0us/step - loss: 2.8947e-04 - val_loss: 6.8521e-05
Epoch 764/1000
32/32 [==============================] - 0s 0us/step - loss: 2.8883e-04 - val_loss: 6.8452e-05
Epoch

Epoch 831/1000
32/32 [==============================] - 0s 0us/step - loss: 2.4944e-04 - val_loss: 6.4445e-05
Epoch 832/1000
32/32 [==============================] - 0s 0us/step - loss: 2.4890e-04 - val_loss: 6.4398e-05
Epoch 833/1000
32/32 [==============================] - 0s 488us/step - loss: 2.4836e-04 - val_loss: 6.4345e-05
Epoch 834/1000
32/32 [==============================] - 0s 0us/step - loss: 2.4782e-04 - val_loss: 6.4297e-05
Epoch 835/1000
32/32 [==============================] - 0s 0us/step - loss: 2.4728e-04 - val_loss: 6.4247e-05
Epoch 836/1000
32/32 [==============================] - 0s 488us/step - loss: 2.4674e-04 - val_loss: 6.4204e-05
Epoch 837/1000
32/32 [==============================] - 0s 0us/step - loss: 2.4622e-04 - val_loss: 6.4147e-05
Epoch 838/1000
32/32 [==============================] - 0s 0us/step - loss: 2.4569e-04 - val_loss: 6.4101e-05
Epoch 839/1000
32/32 [==============================] - 0s 488us/step - loss: 2.4515e-04 - val_loss: 6.4047e-05
Epoc

Epoch 906/1000
32/32 [==============================] - 0s 0us/step - loss: 2.1233e-04 - val_loss: 6.1148e-05
Epoch 907/1000
32/32 [==============================] - 0s 0us/step - loss: 2.1188e-04 - val_loss: 6.1111e-05
Epoch 908/1000
32/32 [==============================] - 0s 0us/step - loss: 2.1142e-04 - val_loss: 6.1072e-05
Epoch 909/1000
32/32 [==============================] - 0s 0us/step - loss: 2.1097e-04 - val_loss: 6.1034e-05
Epoch 910/1000
32/32 [==============================] - 0s 0us/step - loss: 2.1053e-04 - val_loss: 6.0994e-05
Epoch 911/1000
32/32 [==============================] - 0s 0us/step - loss: 2.1008e-04 - val_loss: 6.0957e-05
Epoch 912/1000
32/32 [==============================] - 0s 0us/step - loss: 2.0963e-04 - val_loss: 6.0925e-05
Epoch 913/1000
32/32 [==============================] - 0s 488us/step - loss: 2.0919e-04 - val_loss: 6.0890e-05
Epoch 914/1000
32/32 [==============================] - 0s 0us/step - loss: 2.0874e-04 - val_loss: 6.0855e-05
Epoch 91

Epoch 981/1000
32/32 [==============================] - 0s 489us/step - loss: 1.8120e-04 - val_loss: 5.8729e-05
Epoch 982/1000
32/32 [==============================] - 0s 0us/step - loss: 1.8082e-04 - val_loss: 5.8704e-05
Epoch 983/1000
32/32 [==============================] - 0s 0us/step - loss: 1.8044e-04 - val_loss: 5.8675e-05
Epoch 984/1000
32/32 [==============================] - 0s 487us/step - loss: 1.8007e-04 - val_loss: 5.8644e-05
Epoch 985/1000
32/32 [==============================] - 0s 0us/step - loss: 1.7969e-04 - val_loss: 5.8620e-05
Epoch 986/1000
32/32 [==============================] - 0s 0us/step - loss: 1.7931e-04 - val_loss: 5.8593e-05
Epoch 987/1000
32/32 [==============================] - 0s 0us/step - loss: 1.7894e-04 - val_loss: 5.8561e-05
Epoch 988/1000
32/32 [==============================] - 0s 489us/step - loss: 1.7856e-04 - val_loss: 5.8536e-05
Epoch 989/1000
32/32 [==============================] - 0s 0us/step - loss: 1.7819e-04 - val_loss: 5.8512e-05
Epoc

[array([[ 0.44824603, -0.5362125 ,  0.6257862 ,  0.61135906,  0.50108695],
        [-0.662126  , -0.04732583, -0.51817155, -0.53575623,  0.3642988 ],
        [ 0.8513131 , -1.1041515 ,  0.58935463, -0.8927515 ,  1.5539659 ]],
       dtype=float32),
 array([-0.36749715,  0.00360253, -0.56327033,  0.57296103,  0.6493932 ],
       dtype=float32),
 array([[-0.07041897, -0.04373741, -0.32269704, -0.34918532, -0.13602316,
         -0.3063129 ,  0.01809474,  0.4038335 ,  0.07105578,  0.18572578],
        [ 0.19188567, -0.7061487 , -0.10279872, -0.41128668,  0.06119973,
          0.07042981, -0.3630594 ,  0.40914837, -0.09790229,  0.13569403],
        [ 0.4565307 ,  0.20036274,  0.5966382 , -0.16896828,  0.2800141 ,
         -0.74726164,  0.1998613 , -0.69925684,  0.8198914 , -0.28551838],
        [-0.4366241 , -0.32156065, -0.60621804,  0.7880971 ,  0.42636883,
          0.49007776, -0.78992325,  0.15568699,  0.1818488 ,  0.32909903],
        [-0.3480384 , -0.04211389, -0.51746464,  0.5060566

In [40]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_adam.h5')

In [41]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, sgd, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_21 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_32 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_22 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 6ms/step - loss: 36.169

Epoch 71/1000
32/32 [==============================] - 0s 0us/step - loss: 35.8166 - val_loss: 35.3847
Epoch 72/1000
32/32 [==============================] - 0s 488us/step - loss: 35.8110 - val_loss: 35.3790
Epoch 73/1000
32/32 [==============================] - 0s 0us/step - loss: 35.8053 - val_loss: 35.3733
Epoch 74/1000
32/32 [==============================] - 0s 0us/step - loss: 35.7997 - val_loss: 35.3675
Epoch 75/1000
32/32 [==============================] - 0s 0us/step - loss: 35.7940 - val_loss: 35.3618
Epoch 76/1000
32/32 [==============================] - 0s 0us/step - loss: 35.7884 - val_loss: 35.3561
Epoch 77/1000
32/32 [==============================] - 0s 0us/step - loss: 35.7827 - val_loss: 35.3504
Epoch 78/1000
32/32 [==============================] - 0s 0us/step - loss: 35.7771 - val_loss: 35.3446
Epoch 79/1000
32/32 [==============================] - 0s 487us/step - loss: 35.7714 - val_loss: 35.3389
Epoch 80/1000
32/32 [==============================] - 0s 0us/step - 

Epoch 150/1000
32/32 [==============================] - 0s 0us/step - loss: 35.3725 - val_loss: 34.9353
Epoch 151/1000
32/32 [==============================] - 0s 0us/step - loss: 35.3669 - val_loss: 34.9296
Epoch 152/1000
32/32 [==============================] - 0s 0us/step - loss: 35.3614 - val_loss: 34.9240
Epoch 153/1000
32/32 [==============================] - 0s 488us/step - loss: 35.3558 - val_loss: 34.9183
Epoch 154/1000
32/32 [==============================] - 0s 0us/step - loss: 35.3502 - val_loss: 34.9127
Epoch 155/1000
32/32 [==============================] - 0s 0us/step - loss: 35.3446 - val_loss: 34.9070
Epoch 156/1000
32/32 [==============================] - 0s 0us/step - loss: 35.3390 - val_loss: 34.9014
Epoch 157/1000
32/32 [==============================] - 0s 489us/step - loss: 35.3335 - val_loss: 34.8957
Epoch 158/1000
32/32 [==============================] - 0s 0us/step - loss: 35.3279 - val_loss: 34.8901
Epoch 159/1000
32/32 [==============================] - 0s 0

Epoch 229/1000
32/32 [==============================] - 0s 0us/step - loss: 34.9343 - val_loss: 34.4917
Epoch 230/1000
32/32 [==============================] - 0s 489us/step - loss: 34.9288 - val_loss: 34.4861
Epoch 231/1000
32/32 [==============================] - 0s 0us/step - loss: 34.9233 - val_loss: 34.4805
Epoch 232/1000
32/32 [==============================] - 0s 0us/step - loss: 34.9178 - val_loss: 34.4749
Epoch 233/1000
32/32 [==============================] - 0s 0us/step - loss: 34.9123 - val_loss: 34.4694
Epoch 234/1000
32/32 [==============================] - 0s 0us/step - loss: 34.9067 - val_loss: 34.4638
Epoch 235/1000
32/32 [==============================] - 0s 0us/step - loss: 34.9012 - val_loss: 34.4582
Epoch 236/1000
32/32 [==============================] - 0s 0us/step - loss: 34.8957 - val_loss: 34.4526
Epoch 237/1000
32/32 [==============================] - 0s 489us/step - loss: 34.8902 - val_loss: 34.4471
Epoch 238/1000
32/32 [==============================] - 0s 0

Epoch 308/1000
32/32 [==============================] - 0s 0us/step - loss: 34.5016 - val_loss: 34.0536
Epoch 309/1000
32/32 [==============================] - 0s 0us/step - loss: 34.4962 - val_loss: 34.0481
Epoch 310/1000
32/32 [==============================] - 0s 0us/step - loss: 34.4907 - val_loss: 34.0426
Epoch 311/1000
32/32 [==============================] - 0s 0us/step - loss: 34.4853 - val_loss: 34.0371
Epoch 312/1000
32/32 [==============================] - 0s 0us/step - loss: 34.4798 - val_loss: 34.0316
Epoch 313/1000
32/32 [==============================] - 0s 488us/step - loss: 34.4744 - val_loss: 34.0261
Epoch 314/1000
32/32 [==============================] - 0s 0us/step - loss: 34.4689 - val_loss: 34.0206
Epoch 315/1000
32/32 [==============================] - 0s 0us/step - loss: 34.4635 - val_loss: 34.0151
Epoch 316/1000
32/32 [==============================] - 0s 488us/step - loss: 34.4581 - val_loss: 34.0096
Epoch 317/1000
32/32 [==============================] - 0s 0

...

Epoch 703/1000
32/32 [==============================] - 0s 0us/step - loss: 32.4096 - val_loss: 31.9345
Epoch 704/1000
32/32 [==============================] - 0s 488us/step - loss: 32.4044 - val_loss: 31.9292
Epoch 705/1000
32/32 [==============================] - 0s 0us/step - loss: 32.3992 - val_loss: 31.9240
Epoch 706/1000
32/32 [==============================] - 0s 0us/step - loss: 32.3941 - val_loss: 31.9188
Epoch 707/1000
32/32 [==============================] - 0s 0us/step - loss: 32.3889 - val_loss: 31.9135
Epoch 708/1000
32/32 [==============================] - 0s 0us/step - loss: 32.3837 - val_loss: 31.9083
Epoch 709/1000
32/32 [==============================] - 0s 0us/step - loss: 32.3786 - val_loss: 31.9031
Epoch 710/1000
32/32 [==============================] - 0s 0us/step - loss: 32.3734 - val_loss: 31.8978
Epoch 711/1000
32/32 [==============================] - 0s 488us/step - loss: 32.3683 - val_loss: 31.8926
Epoch 712/1000
32/32 [==============================] - 0s 0

Epoch 782/1000
32/32 [==============================] - 0s 0us/step - loss: 32.0034 - val_loss: 31.5228
Epoch 783/1000
32/32 [==============================] - 0s 0us/step - loss: 31.9983 - val_loss: 31.5176
Epoch 784/1000
32/32 [==============================] - 0s 0us/step - loss: 31.9932 - val_loss: 31.5124
Epoch 785/1000
32/32 [==============================] - 0s 0us/step - loss: 31.9880 - val_loss: 31.5072
Epoch 786/1000
32/32 [==============================] - 0s 0us/step - loss: 31.9829 - val_loss: 31.5020
Epoch 787/1000
32/32 [==============================] - 0s 0us/step - loss: 31.9778 - val_loss: 31.4969
Epoch 788/1000
32/32 [==============================] - 0s 488us/step - loss: 31.9727 - val_loss: 31.4917
Epoch 789/1000
32/32 [==============================] - 0s 0us/step - loss: 31.9676 - val_loss: 31.4865
Epoch 790/1000
32/32 [==============================] - 0s 0us/step - loss: 31.9625 - val_loss: 31.4813
Epoch 791/1000
32/32 [==============================] - 0s 488

Epoch 861/1000
32/32 [==============================] - 0s 488us/step - loss: 31.6007 - val_loss: 31.1146
Epoch 862/1000
32/32 [==============================] - 0s 0us/step - loss: 31.5956 - val_loss: 31.1094
Epoch 863/1000
32/32 [==============================] - 0s 0us/step - loss: 31.5906 - val_loss: 31.1043
Epoch 864/1000
32/32 [==============================] - 0s 0us/step - loss: 31.5855 - val_loss: 31.0991
Epoch 865/1000
32/32 [==============================] - 0s 489us/step - loss: 31.5804 - val_loss: 31.0940
Epoch 866/1000
32/32 [==============================] - 0s 0us/step - loss: 31.5753 - val_loss: 31.0888
Epoch 867/1000
32/32 [==============================] - 0s 0us/step - loss: 31.5703 - val_loss: 31.0837
Epoch 868/1000
32/32 [==============================] - 0s 487us/step - loss: 31.5652 - val_loss: 31.0786
Epoch 869/1000
32/32 [==============================] - 0s 0us/step - loss: 31.5601 - val_loss: 31.0734
Epoch 870/1000
32/32 [==============================] - 0s

Epoch 940/1000
32/32 [==============================] - 0s 489us/step - loss: 31.2013 - val_loss: 30.7096
Epoch 941/1000
32/32 [==============================] - 0s 0us/step - loss: 31.1962 - val_loss: 30.7045
Epoch 942/1000
32/32 [==============================] - 0s 0us/step - loss: 31.1912 - val_loss: 30.6994
Epoch 943/1000
32/32 [==============================] - 0s 0us/step - loss: 31.1862 - val_loss: 30.6943
Epoch 944/1000
32/32 [==============================] - 0s 488us/step - loss: 31.1811 - val_loss: 30.6892
Epoch 945/1000
32/32 [==============================] - 0s 0us/step - loss: 31.1761 - val_loss: 30.6841
Epoch 946/1000
32/32 [==============================] - 0s 0us/step - loss: 31.1711 - val_loss: 30.6790
Epoch 947/1000
32/32 [==============================] - 0s 487us/step - loss: 31.1660 - val_loss: 30.6739
Epoch 948/1000
32/32 [==============================] - 0s 0us/step - loss: 31.1610 - val_loss: 30.6688
Epoch 949/1000
32/32 [==============================] - 0s

[array([[ 0.27329165, -0.13302097,  0.2653236 ,  0.19208816, -0.5837717 ],
        [ 0.10231543,  0.7499549 ,  0.23936205,  0.5341293 ,  0.75497675],
        [ 0.72581214, -0.79050475,  0.36488077, -0.24614193,  0.17588478]],
       dtype=float32),
 array([-0.02444298, -0.01294862,  0.0406203 ,  0.00543027,  0.01965185],
       dtype=float32),
 array([[ 0.13448219,  0.4066646 ,  0.33047602, -0.53340364, -0.14590316,
         -0.61134714, -0.60923195,  0.47031373,  0.33777714, -0.488678  ],
        [ 0.27700388, -0.22296569, -0.02444633, -0.26447397, -0.00203225,
          0.22206406,  0.1464017 , -0.3197855 , -0.16842839, -0.17485476],
        [-0.43849042, -0.21041742, -0.19037223,  0.02997789, -0.41673768,
         -0.25269935,  0.05500212,  0.29766387,  0.23217916,  0.00433868],
        [ 0.4229257 , -0.30426705, -0.28446198,  0.00295528,  0.59689504,
         -0.10025524, -0.23799136, -0.02643682, -0.16870685, -0.47045293],
        [ 0.30463013, -0.4159491 , -0.10106722,  0.2195770

In [42]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_sgd.h5')

In [43]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, RMSprop, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_23 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_35 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_24 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 7ms/step - loss: 43.233

Epoch 72/1000
32/32 [==============================] - 0s 488us/step - loss: 0.0883 - val_loss: 0.0399
Epoch 73/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0870 - val_loss: 0.0284
Epoch 74/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0795 - val_loss: 0.0304
Epoch 75/1000
32/32 [==============================] - 0s 489us/step - loss: 0.0721 - val_loss: 0.0204
Epoch 76/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0657 - val_loss: 0.0240
Epoch 77/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0619 - val_loss: 0.0170
Epoch 78/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0593 - val_loss: 0.0221
Epoch 79/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0582 - val_loss: 0.0163
Epoch 80/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0580 - val_loss: 0.0237
Epoch 81/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0592 - val

Epoch 153/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0634 - val_loss: 0.0330
Epoch 154/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0571 - val_loss: 0.0134
Epoch 155/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0458 - val_loss: 0.0213
Epoch 156/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0384 - val_loss: 0.0086
Epoch 157/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0332 - val_loss: 0.0165
Epoch 158/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0306 - val_loss: 0.0063
Epoch 159/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0294 - val_loss: 0.0187
Epoch 160/1000
32/32 [==============================] - 0s 488us/step - loss: 0.0301 - val_loss: 0.0057
Epoch 161/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0338 - val_loss: 0.0444
Epoch 162/1000
32/32 [==============================] - 0s 0us/step - loss: 0.04

Epoch 233/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0808 - val_loss: 0.0213
Epoch 234/1000
32/32 [==============================] - 0s 489us/step - loss: 0.0581 - val_loss: 0.0426
Epoch 235/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0487 - val_loss: 0.0094
Epoch 236/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0362 - val_loss: 0.0244
Epoch 237/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0314 - val_loss: 0.0093
Epoch 238/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0274 - val_loss: 0.0202
Epoch 239/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0265 - val_loss: 0.0136
Epoch 240/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0266 - val_loss: 0.0221
Epoch 241/1000
32/32 [==============================] - 0s 487us/step - loss: 0.0284 - val_loss: 0.0213
Epoch 242/1000
32/32 [==============================] - 0s 0us/step - loss: 0.

Epoch 313/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0204 - val_loss: 0.0222
Epoch 314/1000
32/32 [==============================] - 0s 488us/step - loss: 0.0212 - val_loss: 0.0185
Epoch 315/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0242 - val_loss: 0.0445
Epoch 316/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0335 - val_loss: 0.0394
Epoch 317/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0513 - val_loss: 0.0939
Epoch 318/1000
32/32 [==============================] - 0s 488us/step - loss: 0.0774 - val_loss: 0.0339
Epoch 319/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0701 - val_loss: 0.0589
Epoch 320/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0611 - val_loss: 0.0090
Epoch 321/1000
32/32 [==============================] - 0s 488us/step - loss: 0.0404 - val_loss: 0.0303
Epoch 322/1000
32/32 [==============================] - 0s 0us/step - loss: 

...

Epoch 713/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0307 - val_loss: 0.0459
Epoch 714/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0430 - val_loss: 0.0337
Epoch 715/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0440 - val_loss: 0.0429
Epoch 716/1000
32/32 [==============================] - 0s 488us/step - loss: 0.0424 - val_loss: 0.0224
Epoch 717/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0283 - val_loss: 0.0226
Epoch 718/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0202 - val_loss: 0.0115
Epoch 719/1000
32/32 [==============================] - 0s 488us/step - loss: 0.0135 - val_loss: 0.0140
Epoch 720/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0106 - val_loss: 0.0083
Epoch 721/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0089 - val_loss: 0.0133
Epoch 722/1000
32/32 [==============================] - 0s 0us/step - loss: 0.

Epoch 793/1000
32/32 [==============================] - 0s 488us/step - loss: 0.0190 - val_loss: 0.0146
Epoch 794/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0170 - val_loss: 0.0062
Epoch 795/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0149 - val_loss: 0.0128
Epoch 796/1000
32/32 [==============================] - 0s 488us/step - loss: 0.0129 - val_loss: 0.0052
Epoch 797/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0118 - val_loss: 0.0148
Epoch 798/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0135 - val_loss: 0.0103
Epoch 799/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0171 - val_loss: 0.0298
Epoch 800/1000
32/32 [==============================] - 0s 488us/step - loss: 0.0265 - val_loss: 0.0289
Epoch 801/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0372 - val_loss: 0.0522
Epoch 802/1000
32/32 [==============================] - 0s 0us/step - loss: 

Epoch 873/1000
32/32 [==============================] - 0s 488us/step - loss: 0.0115 - val_loss: 0.0123
Epoch 874/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0135 - val_loss: 0.0080
Epoch 875/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0153 - val_loss: 0.0176
Epoch 876/1000
32/32 [==============================] - 0s 488us/step - loss: 0.0195 - val_loss: 0.0121
Epoch 877/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0230 - val_loss: 0.0250
Epoch 878/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0286 - val_loss: 0.0147
Epoch 879/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0275 - val_loss: 0.0226
Epoch 880/1000
32/32 [==============================] - 0s 488us/step - loss: 0.0249 - val_loss: 0.0100
Epoch 881/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0183 - val_loss: 0.0159
Epoch 882/1000
32/32 [==============================] - 0s 0us/step - loss: 

Epoch 953/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0159 - val_loss: 0.0101
Epoch 954/1000
32/32 [==============================] - 0s 488us/step - loss: 0.0137 - val_loss: 0.0044
Epoch 955/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0111 - val_loss: 0.0083
Epoch 956/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0105 - val_loss: 0.0044
Epoch 957/1000
32/32 [==============================] - 0s 488us/step - loss: 0.0097 - val_loss: 0.0090
Epoch 958/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0104 - val_loss: 0.0056
Epoch 959/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0110 - val_loss: 0.0120
Epoch 960/1000
32/32 [==============================] - 0s 488us/step - loss: 0.0133 - val_loss: 0.0085
Epoch 961/1000
32/32 [==============================] - 0s 0us/step - loss: 0.0157 - val_loss: 0.0187
Epoch 962/1000
32/32 [==============================] - 0s 0us/step - loss: 

[array([[-0.16869327, -0.7028091 ,  0.19091664, -0.05091389, -0.42100996],
        [-0.10379342,  0.45014456, -0.38592345, -0.04836174,  0.32234126],
        [-0.6189677 ,  0.7206661 , -0.37803945,  0.02417222, -0.23914549]],
       dtype=float32),
 array([-3.5920426e-01, -9.2068844e-02,  4.0622973e-01, -3.3739858e-04,
         3.2936671e-01], dtype=float32),
 array([[ 0.0029519 ,  0.38498607,  0.14179392, -0.45090765, -0.6728841 ,
         -0.77447635,  0.8127178 ,  0.576019  , -0.2365474 ,  0.5149411 ],
        [-0.432804  ,  0.07383288,  0.35399702,  0.11030995, -0.38934827,
         -0.26872385,  0.02268389, -0.19497088,  0.02194052,  0.314507  ],
        [ 0.38337153, -0.30797952, -0.05082429,  0.6944473 ,  0.3157798 ,
          0.5046    , -0.41499552, -0.08126837, -0.18509294,  0.12852567],
        [-0.30005515, -0.02371627, -0.22033378,  0.5938354 , -0.3041826 ,
         -0.18783678, -0.47074306, -0.6765789 ,  0.04879604,  0.15810741],
        [ 0.5483183 , -0.3296121 ,  0.9410

In [44]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_rmsprop.h5')